(chapter-cwic)=
# Use Case - Granule Search (CWIC/CMR)

This chapter provides a comprehensive and detailed process about how to implement a WGISS
OpenSearch client, which includes how to retrieve the IDN collection ID for the collection of
interest, and how to build an OpenSearch request for a CWIC data partner. Within CMR, CWIC
partners are available directly from CMR. The IDN response records contain direct links to the
OSDD from the CWIC partner server, where possible. This can be used to construct a granule
search directly from the partner.

In [22]:
import re
import json, requests, xml
# import pandas as pd

from xml.dom import minidom
from xml.etree import ElementTree
from IPython.display import Markdown as md
# from IPython.display import HTML as html

In [23]:
def get_api_request(template, os_querystring):
  #
  # Fill (URL) template with OpenSearch parameter values provided in os_querystring and 
  # return as short HTTP URL without empty parameters.
  #
  
  # Limitation: the OSDD may use a default namespace for OpenSearch instead of using "os".
  # We make a simple correction here allowing to use OpenSearch queryables without namespace in requests.
  # A more generic solution to obtain namespaces from the OSDD and compare them with user supplied namespaces is future work.
  
  OS_NAMESPACE = 'os:'
      
  # perform substitutions in template
  for p in os_querystring:
      # print("  .. replacing:", p, "by", os_querystring[p])
      # template = re.sub('\{'+p+'.*?\}', os_querystring[p] , template)
      result = re.subn('\{'+p+'.*?\}', os_querystring[p] , template)
      n = result[1]
      template = result[0]
      if (n<1):
          if (':' in p):
                print("ERROR: parameter " + p + " not found in template.")
          else:
                # try with explicit namespace
                result = re.subn('\{'+OS_NAMESPACE+p+'.*?\}', os_querystring[p] , template)
                n = result[1]
                template = result[0]
                if (n<1):
                    print("ERROR: parameter " + OS_NAMESPACE+p + " not found in template.")   
      
      # print("- intermediate new template:" + template)
      
  # remove empty search parameters
  template=re.sub('&?[a-zA-Z]*=\{.*?\}', '' , template)
  
  # remove remaining empty search parameters which did not have an HTTP query parameter attached (e.g. /{time:end}).
  template=re.sub('.?\{.*?\}', '' , template)
  
  # print("API request: " + template)
            
  return (template)

## Retrieve Collection OSDD via IDN OpenSearch

The steps in this section are identical to the steps explained in the section [Retrieve Collections via IDN OpenSearch](section_Retrieve_Collections_via_IDN_OpenSearch). To fit the end-to-end
CWIC use case, the parameter values for the collection search has been selected in such a way that
the granule search is eventually performed through CWIC.


| **Use Case Overview**  |  | 
| -------- | --------- | 
| Title | Clients start from the IDN OpenSearch | 
| Description | This use case describes steps for retrieving a collection ID from the IDN OpenSearch. | 
| Actors | OpenSearch client, IDN OpenSearch Server | 
| Initial Status and Preconditions | Clients have the IDN OpenSearch access URL. | 

The following steps describe this use case.

**Step 1**  
>  Obtain the IDN OpenSearch OSDD to formulate a valid IDN OpenSearch request.

In [24]:
url_osdd = "https://cmr.earthdata.nasa.gov/opensearch/collections/descriptor_document.xml?clientId=ceosOpenSearchDoc"

In [25]:
md("The OpenSearch Description Document is accessible at the fixed location [{url}]({url}) and contains the URL template to be used for collection search.".format(url=url_osdd))

The OpenSearch Description Document is accessible at the fixed location [https://cmr.earthdata.nasa.gov/opensearch/collections/descriptor_document.xml?clientId=ceosOpenSearchDoc](https://cmr.earthdata.nasa.gov/opensearch/collections/descriptor_document.xml?clientId=ceosOpenSearchDoc) and contains the URL template to be used for collection search.

In [26]:
response = requests.get( url_osdd )

xmlstr = minidom.parseString(response.text).toprettyxml(indent='  ',newl='')
# md("```xml\n" + xmlstr + "\n```\n")

Find the URL template for collection search with Atom responses.

In [27]:
root = ElementTree.fromstring(response.text)

ns = {'os': 'http://a9.com/-/spec/opensearch/1.1/'}
collection_url_atom = root.find('os:Url[@rel="collection"][@type="application/atom+xml"]', ns)

collection_template = collection_url_atom.attrib['template']
collection_template

'https://cmr.earthdata.nasa.gov/opensearch/collections.atom?keyword={os:searchTerms?}&instrument={echo:instrument?}&satellite={eo:platform?}&boundingBox={geo:box?}&lat={geo:lat?}&lon={geo:lon?}&radius={geo:radius?}&geometry={geo:geometry?}&placeName={geo:name?}&startTime={time:start?}&endTime={time:end?}&cursor={os:startPage?}&numberOfResults={os:count?}&offset={os:startIndex?}&uid={geo:uid?}&hasGranules={echo:hasGranules?}&isCwic={echo:isCwic?}&isGeoss={echo:isGeoss?}&isCeos={echo:isCeos?}&isEosdis={echo:isEosdis?}&isFedeo={echo:isFedeo?}&provider={echo:provider?}&clientId=ceosOpenSearchDoc'

**Step 2**  
>  Search collections of interest through IDN OpenSearch with proper request parameters (e.g. spatial footprint, temporal extent and keyword). A complete list of supported request parameters, extracted from the IDN OpenSearch OSDD, was included in the section [Available Collection Search Criteria](section_Available_Collection_Search_Criteria_IDN).

An example request can be formed as follows.

In [28]:
# request_url = get_api_request(collection_template, {'count': '1', 'searchTerms': 'Landsat_8'})
request_url = get_api_request(collection_template, {'count': '1', 'searchTerms': 'GEDI L4A Footprint'})
request_url

'https://cmr.earthdata.nasa.gov/opensearch/collections.atom?keyword=GEDI L4A Footprint&numberOfResults=1&clientId=ceosOpenSearchDoc'

In [29]:
response = requests.get( request_url )
xmlstr = minidom.parseString(response.text).toprettyxml(indent='   ', newl='')
md("```xml\n" + xmlstr + "\n```\n")

```xml
<?xml version="1.0" ?><feed xmlns:relevance="http://a9.com/-/opensearch/extensions/relevance/1.0/" xmlns:os="http://a9.com/-/spec/opensearch/1.1/" xmlns:georss="http://www.georss.org/georss" xmlns="http://www.w3.org/2005/Atom" xmlns:dc="http://purl.org/dc/elements/1.1/" xmlns:echo="https://cmr.earthdata.nasa.gov/search/site/docs/search/api.html#atom" xmlns:esipdiscovery="http://commons.esipfed.org/ns/discovery/1.2/" xmlns:gml="http://www.opengis.net/gml" xmlns:time="http://a9.com/-/opensearch/extensions/time/1.0/" xmlns:eo="http://a9.com/-/opensearch/extensions/eo/1.0/" esipdiscovery:version="1.2">   
     <updated>2025-07-08T14:56:06.299Z</updated>   
     <id>https://cmr.earthdata.nasa.gov/opensearch/collections.atom</id>   
     <author>      
          <name>CMR</name>      
          <email>echodev@echo.nasa.gov</email>      
     </author>   
     <title type="text">ECHO dataset metadata</title>   
     <subtitle type="text">Search parameters: keyword =&gt; GEDI L4A Footprint</subtitle>   
     <link href="https://cmr.earthdata.nasa.gov/opensearch/granules/descriptor_document.xml" hreflang="en-US" type="application/opensearchdescription+xml" rel="search"/>   
     <link href="https://cmr.earthdata.nasa.gov/opensearch/collections.atom?keyword=GEDI%20L4A%20Footprint&amp;numberOfResults=1&amp;clientId=ceosOpenSearchDoc" hreflang="en-US" type="application/atom+xml" rel="self"/>   
     <link href="https://cmr.earthdata.nasa.gov/opensearch/collections.atom?keyword=GEDI%20L4A%20Footprint&amp;numberOfResults=1&amp;clientId=ceosOpenSearchDoc&amp;cursor=7" hreflang="en-US" type="application/atom+xml" rel="last"/>   
     <link href="https://cmr.earthdata.nasa.gov/opensearch/collections.atom?keyword=GEDI%20L4A%20Footprint&amp;numberOfResults=1&amp;clientId=ceosOpenSearchDoc&amp;cursor=2" hreflang="en-US" type="application/atom+xml" rel="next"/>   
     <link href="https://cmr.earthdata.nasa.gov/opensearch/collections.atom?keyword=GEDI%20L4A%20Footprint&amp;numberOfResults=1&amp;clientId=ceosOpenSearchDoc&amp;cursor=1" hreflang="en-US" type="application/atom+xml" rel="first"/>   
     <link href="https://wiki.earthdata.nasa.gov/display/echo/Open+Search+API+release+information" hreflang="en-US" type="text/html" rel="describedBy" title="Release Notes"/>   
     <os:Query xmlns:geo="http://a9.com/-/opensearch/extensions/geo/1.0/" role="request" os:searchTerms="GEDI L4A Footprint"/>   
     <os:totalResults>6</os:totalResults>   
     <os:itemsPerPage>1</os:itemsPerPage>   
     <os:startIndex>1</os:startIndex>   
     <entry>      
          <id>https://cmr.earthdata.nasa.gov/opensearch/collections.atom?uid=C2237824918-ORNL_CLOUD</id>      
          <author>         
               <name>CMR</name>         
               <email>echodev@echo.nasa.gov</email>         
          </author>      
          <consortium>GEOSS</consortium>      
          <consortium>EOSDIS</consortium>      
          <title type="text">GEDI L4A Footprint Level Aboveground Biomass Density, Version 2.1</title>      
          <summary type="text">This dataset contains Global Ecosystem Dynamics Investigation (GEDI) Level 4A (L4A) Version 2 predictions of the aboveground biomass density (AGBD; in Mg/ha) and estimates of the prediction standard error within each sampled geolocated laser footprint. In this version, the granules are in sub-orbits. The algorithm setting group selection used for GEDI02_A Version 2 has been modified for Evergreen Broadleaf Trees in South America to reduce false positive errors resulting from the selection of waveform modes above ground elevation as the lowest mode. The footprints are located within the global latitude band observed by the International Space Station (ISS), nominally 51.6 degrees N and S and reported for the period 2019-04-18 to 2024-11-27. No acquisitions occurred while the GEDI instrument was in storage on the International Space Station (ISS) from March 2023 to April 2024. The GEDI instrument consists of three lasers producing a total of eight beam ground transects, which instantaneously sample eight ~25 m footprints spaced approximately every 60 m along-track. The GEDI beam transects are spaced approximately 600 m apart on the Earth's surface in the cross-track direction, for an across-track width of ~4.2 km. Footprint AGBD was derived from parametric models that relate simulated GEDI Level 2A (L2A) waveform relative height (RH) metrics to field plot estimates of AGBD. Height metrics from simulated waveforms associated with field estimates of AGBD from multiple regions and plant functional types (PFTs) were compiled to generate a calibration dataset for models representing the combinations of world regions and PFTs (i.e., deciduous broadleaf trees, evergreen broadleaf trees, evergreen needleleaf trees, deciduous needleleaf trees, and the combination of grasslands, shrubs, and woodlands). For each of the eight beams, additional data are reported with the AGBD estimates, including the associated uncertainty metrics, quality flags, model inputs, and other information about the GEDI L2A waveform for this selected algorithm setting group. Model inputs include the scaled and transformed GEDI L2A RH metrics, footprint geolocation variables and land cover input data including PFTs and the world region identifiers. Additional model outputs include the AGBD predictions for each of the six GEDI L2A algorithm setting groups with AGBD in natural and transformed units and associated prediction uncertainty for each GEDI L2A algorithm setting group. Providing these ancillary data products will allow users to evaluate and select alternative algorithm setting groups. Also provided are outputs of parameters and variables from the L4A models used to generate AGBD predictions that are required as input to the GEDI04_B algorithm to generate 1-km gridded products.</summary>      
          <updated>2025-04-14T22:32:09.000Z</updated>      
          <link rel="enclosure" hreflang="en-US" href="https://search.earthdata.nasa.gov/search?q=GEDI_L4A_AGB_Density_V2_1_2056&amp;ac=true"/>      
          <link rel="describedBy" hreflang="en-US" href="https://daac.ornl.gov/GEDI/guides/GEDI_L4A_AGB_Density_V2_1.html" type="text/html"/>      
          <link rel="via" hreflang="en-US" href="https://doi.org/10.3334/ORNLDAAC/2056"/>      
          <link rel="http://esipfed.org/ns/fedsearch/1.1/service#" hreflang="en-US" href="https://opendap.earthdata.nasa.gov/"/>      
          <link rel="icon" hreflang="en-US" href="https://daac.ornl.gov/GEDI/guides/GEDI_L4A_AGB_Density_V2_1_Fig1.png" type="image/png"/>      
          <link rel="via" hreflang="en-US" href="https://gedi.umd.edu"/>      
          <link rel="describedBy" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_ATBD_L4A_v1.0.pdf" type="application/pdf"/>      
          <link rel="describedBy" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_L4A_V2_Common_Queries.pdf" type="application/pdf"/>      
          <link rel="describedBy" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_L4A_AGB_Density_V2_1.pdf" type="application/pdf"/>      
          <link rel="describedBy" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_deleted_files_mw_163_182.txt"/>      
          <link rel="describedBy" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_revised_files_mw_163_182.txt"/>      
          <link rel="describedBy" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_deleted_files_orbits_17591_09899.txt"/>      
          <link rel="describedBy" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_L4A_V2_Product_Data_Dictionary.pdf" type="application/pdf"/>      
          <link rel="describedBy" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_revised_files_orbits_17591_09899.txt"/>      
          <link href="https://cmr.earthdata.nasa.gov/opensearch/granules.atom?clientId=ceosOpenSearchDoc&amp;shortName=GEDI_L4A_AGB_Density_V2_1_2056&amp;versionId=2.1&amp;dataCenter=ORNL_CLOUD" hreflang="en-US" type="application/atom+xml" rel="search" title="Search for granules"/>      
          <link href="https://cmr.earthdata.nasa.gov/opensearch/granules/descriptor_document.xml?collectionConceptId=C2237824918-ORNL_CLOUD&amp;clientId=ceosOpenSearchDoc" hreflang="en-US" type="application/opensearchdescription+xml" rel="search" title="Granule OpenSearch Descriptor Document"/>      
          <link href="https://cmr.earthdata.nasa.gov/search/concepts/C2237824918-ORNL_CLOUD.xml" hreflang="en-US" type="application/xml" rel="via" title="Product metadata"/>      
          <dc:identifier>C2237824918-ORNL_CLOUD</dc:identifier>      
          <dc:date>2019-04-17T00:00:00.000Z/2024-11-27T23:59:59.999Z</dc:date>      
          <echo:datasetId>GEDI L4A Footprint Level Aboveground Biomass Density, Version 2.1</echo:datasetId>      
          <echo:shortName>GEDI_L4A_AGB_Density_V2_1_2056</echo:shortName>      
          <echo:versionId>2.1</echo:versionId>      
          <echo:entryId>GEDI_L4A_AGB_Density_V2_1_2056_2.1</echo:entryId>      
          <echo:dataCenter>ORNL_CLOUD</echo:dataCenter>      
          <echo:archiveCenter>ORNL_DAAC</echo:archiveCenter>      
          <echo:organization>ORNL_DAAC</echo:organization>      
          <echo:processingLevelId>4</echo:processingLevelId>      
          <echo:coordinateSystem>CARTESIAN</echo:coordinateSystem>      
          <echo:orbitParameters/>      
          <georss:box>-53 -180 55.7983 180</georss:box>      
          <echo:hasVariables>true</echo:hasVariables>      
          <echo:hasFormats>true</echo:hasFormats>      
          <echo:hasTransforms>false</echo:hasTransforms>      
          <echo:hasCombine>false</echo:hasCombine>      
          <echo:hasSpatialSubsetting>true</echo:hasSpatialSubsetting>      
          <echo:hasTemporalSubsetting>true</echo:hasTemporalSubsetting>      
          <echo:cloudHosted>true</echo:cloudHosted>      
          <relevance:score>1.32</relevance:score>      
          <echo:tag>         
               <echo:tagKey>gov.nasa.eosdis</echo:tagKey>         
          </echo:tag>      
          <echo:is_eosdis>true</echo:is_eosdis>      
     </entry>   
</feed>
```


**Step 3**  
>  From the IDN OpenSearch response, obtain the OSDD endpoint for the collection by parsing the href attribute under `<link rel="search" type=”application/opensearchdescription+xml” >` 

In [30]:
root = ElementTree.fromstring(response.text)
# Extract <link> element with the OSDD for the granule search
el = root.find('{http://www.w3.org/2005/Atom}entry/{http://www.w3.org/2005/Atom}link[@rel="search"][@type="application/opensearchdescription+xml"]')
xmltxt = ElementTree.tostring(el, encoding='unicode', method='xml')
md("```xml\n" + xmltxt + "\n```\n")

```xml
<ns0:link xmlns:ns0="http://www.w3.org/2005/Atom" href="https://cmr.earthdata.nasa.gov/opensearch/granules/descriptor_document.xml?collectionConceptId=C2237824918-ORNL_CLOUD&amp;clientId=ceosOpenSearchDoc" hreflang="en-US" type="application/opensearchdescription+xml" rel="search" title="Granule OpenSearch Descriptor Document" />
    
```


In [31]:
url_osdd = el.attrib['href']
el.attrib['href']

'https://cmr.earthdata.nasa.gov/opensearch/granules/descriptor_document.xml?collectionConceptId=C2237824918-ORNL_CLOUD&clientId=ceosOpenSearchDoc'

In [32]:
# work-around
# Extract the short name for future use.  Should not be needed according to CEOS OpenSearch Best Practices.  
el = root.find('{http://www.w3.org/2005/Atom}entry/{https://cmr.earthdata.nasa.gov/search/site/docs/search/api.html#atom}shortName')
el.text
short_name = el.text

This response will provide the key component of searching CMR for the desired collection using
the CMR conceptID. For Landsat 8, this will be C1235542031-USGS_LTA. Now obtain from
CMR the URL for the local OSDD (in this case, at USGS), with the request

Retrieve the response (i.e. the OSDD document).

In [33]:
response = requests.get( url_osdd )

xmlstr = minidom.parseString(response.text).toprettyxml(indent='  ',newl='')
md("```xml\n" + xmlstr + "\n```\n")


```xml
<?xml version="1.0" ?><feed xmlns="http://a9.com/-/spec/opensearch/1.1/" xmlns:geo="http://a9.com/-/opensearch/extensions/geo/1.0/" xmlns:params="http://a9.com/-/spec/opensearch/extensions/paramseters/1.0/" xmlns:time="http://a9.com/-/opensearch/extensions/time/1.0/" xmlns:eop="http://a9.com/-/opensearch/extensions/eo/1.0/">  
    <title>OpenSearch Exception</title>  
    <subtitle type="text">No provider present within the `org.ceos.wgiss.cwic.granules.provider` tag.</subtitle>  
    <updated>2025-07-08T14:56:08Z</updated>  
    <id>https://cmr.earthdata.nasa.gov/opensearch/granules/descriptor_document.xml?collectionConceptId=C2237824918-ORNL_CLOUD&amp;clientId=ceosOpenSearchDoc</id>  
    <link title="self" rel="self" type="application/atom+xml" href="https://cmr.earthdata.nasa.gov/opensearch/granules/descriptor_document.xml?collectionConceptId=C2237824918-ORNL_CLOUD&amp;clientId=ceosOpenSearchDoc"/>  
    <totalResults>0</totalResults>  
</feed>
```


```{warning}
In the previous step, the OSDD could not be retrieved as the URL included in the response is not the one with `short_name`.  See also step 5 in chapter 5 "Outline" with the workaround to fix that.
```

In [34]:
root = ElementTree.fromstring(response.text)

try:
    el = root.find('{http://a9.com/-/spec/opensearch/1.1/}title')
    print(el.text)

    if (el.text == 'OpenSearch Exception'):
        url_osdd = "https://cmr.earthdata.nasa.gov/opensearch/granules/descriptor_document.xml?shortName=" + short_name
        print("Corrected OSDD URL: " + url_osdd)
except:
    pass

OpenSearch Exception
Corrected OSDD URL: https://cmr.earthdata.nasa.gov/opensearch/granules/descriptor_document.xml?shortName=GEDI_L4A_AGB_Density_V2_1_2056


In [35]:
url_osdd

'https://cmr.earthdata.nasa.gov/opensearch/granules/descriptor_document.xml?shortName=GEDI_L4A_AGB_Density_V2_1_2056'

In the response:

In [36]:
# Get the OSDD document for granule search
response = requests.get( url_osdd )

xmlstr = minidom.parseString(response.text).toprettyxml(indent='  ',newl='')
md("```xml\n" + xmlstr + "\n```\n")

```xml
<?xml version="1.0" ?><os:OpenSearchDescription xmlns:os="http://a9.com/-/spec/opensearch/1.1/" xmlns:echo="https://cmr.earthdata.nasa.gov/search/site/docs/search/api.html#atom" xmlns:geo="http://a9.com/-/opensearch/extensions/geo/1.0/" xmlns:time="http://a9.com/-/opensearch/extensions/time/1.0/" xmlns:params="http://a9.com/-/spec/opensearch/extensions/parameters/1.0/" xmlns:referrer="http://www.opensearch.org/Specifications/OpenSearch/Extensions/Referrer/1.0" xmlns:eo="http://a9.com/-/opensearch/extensions/eo/1.0/" xmlns:atom="http://www.w3.org/2005/Atom">  
    <os:ShortName>CMR Granules</os:ShortName>  
    <os:Description>NASA CMR Granule search using geo, time and parameter extensions</os:Description>  
    <os:Contact>echodev@echo.nasa.gov</os:Contact>  

    <os:Url type="application/atom+xml" rel="results" params:method="GET" template="https://cmr.earthdata.nasa.gov/opensearch/granules.atom?datasetId={echo:datasetId?}&amp;shortName=GEDI_L4A_AGB_Density_V2_1_2056&amp;versionId={echo:versionId?}&amp;dataCenter={echo:dataCenter?}&amp;boundingBox={geo:box?}&amp;lat={geo:lat?}&amp;lon={geo:lon?}&amp;radius={geo:radius?}&amp;geometry={geo:geometry?}&amp;placeName={geo:name?}&amp;startTime={time:start?}&amp;endTime={time:end?}&amp;cursor={os:startPage?}&amp;numberOfResults={os:count?}&amp;offset={os:startIndex?}&amp;uid={geo:uid?}&amp;parentIdentifier={eo:parentIdentifier?}">    
        <params:Parameter name="datasetId" uiDisplay="Collection identifier" value="{echo:datasetId}" title="Inventory associated with a dataset expressed as an ID" minimum="0"/>    
        <params:Parameter name="boundingBox" uiDisplay="Bounding box" value="{geo:box}" title="Inventory with a spatial extent overlapping this bounding box" minimum="0"/>    
        <params:Parameter name="lat" uiDisplay="Latitude" value="{geo:lat}" title="Inventory with latitude in decimal degrees, must be used together with lon and radius" minimum="0" minInclusive="-90.0" maxInclusive="90.0"/>    
        <params:Parameter name="lon" uiDisplay="Longitude" value="{geo:lon}" title="Inventory with longitude in decimal degrees, must be used together with lat and radius" minimum="0" minInclusive="-180.0" maxInclusive="180.0"/>    
        <params:Parameter name="radius" uiDisplay="Radius" value="{geo:radius}" title="Inventory with the search radius in meters, must be used together with lat and lon" minimum="0" minInclusive="10" maxInclusive="6000000"/>    
        <params:Parameter name="geometry" uiDisplay="Geometry" value="{geo:geometry}" title="Inventory with a spatial extent overlapping this geometry" minimum="0">      
            <atom:link rel="profile" href="http://www.opengis.net/wkt/LINESTRING" title="This service accepts WKT LineStrings"/>      
            <atom:link rel="profile" href="http://www.opengis.net/wkt/POINT" title="This service accepts WKT Points"/>      
            <atom:link rel="profile" href="http://www.opengis.net/wkt/POLYGON" title="This service accepts WKT Polygons"/>      
        </params:Parameter>    
        <params:Parameter name="placeName" uiDisplay="Place name" value="{geo:name}" title="Inventory with a spatial location described by this name" minimum="0"/>    
        <params:Parameter name="startTime" uiDisplay="Start time" value="{time:start}" title="Inventory with a temporal extent containing this start time" minimum="0"/>    
        <params:Parameter name="endTime" uiDisplay="End time" value="{time:end}" title="Inventory with a temporal extent containing this end time" minimum="0"/>    
        <params:Parameter name="cursor" uiDisplay="Start page" value="{os:startPage}" title="Start page for the search result" minimum="0"/>    
        <params:Parameter name="numberOfResults" uiDisplay="Number of results" value="{os:count}" title="Maximum number of records in the search result" minimum="0" maxInclusive="2000"/>    
        <params:Parameter name="offset" uiDisplay="Start index" value="{os:startIndex}" title="0-based offset used to skip the specified number of results in the search result set" minimum="0"/>    
        <params:Parameter name="uid" uiDisplay="Unique identifier identifier" value="{geo:uid}" title="Inventory associated with this unique ID" minimum="0"/>    
        <params:Parameter name="parentIdentifier" uiDisplay="CMR Collection concept identifier" value="{eo:parentIdentifier}" title="Inventory associated with a dataset expressed as a CMR concept ID" minimum="0"/>    
        <params:Parameter name="clientId" uiDisplay="Client identifier" value="{referrer:source}" title="Client identifier to be used for metrics" minimum="0"/>    
    </os:Url>  

    <os:Url type="text/html" rel="results" params:method="GET" template="https://cmr.earthdata.nasa.gov/opensearch/granules.html?datasetId={echo:datasetId?}&amp;shortName=GEDI_L4A_AGB_Density_V2_1_2056&amp;versionId={echo:versionId?}&amp;dataCenter={echo:dataCenter?}&amp;boundingBox={geo:box?}&amp;lat={geo:lat?}&amp;lon={geo:lon?}&amp;radius={geo:radius?}&amp;geometry={geo:geometry?}&amp;placeName={geo:name?}&amp;startTime={time:start?}&amp;endTime={time:end?}&amp;cursor={os:startPage?}&amp;numberOfResults={os:count?}&amp;offset={os:startIndex?}&amp;uid={geo:uid?}&amp;parentIdentifier={eo:parentIdentifier?}">    
        <params:Parameter name="datasetId" uiDisplay="Collection identifier" value="{echo:datasetId}" title="Inventory associated with a dataset expressed as an ID" minimum="0"/>    
        <params:Parameter name="boundingBox" uiDisplay="Bounding box" value="{geo:box}" title="Inventory with a spatial extent overlapping this bounding box" minimum="0"/>    
        <params:Parameter name="lat" uiDisplay="Latitude" value="{geo:lat}" title="Inventory with latitude in decimal degrees, must be used together with lon and radius" minimum="0" minInclusive="-90.0" maxInclusive="90.0"/>    
        <params:Parameter name="lon" uiDisplay="Longitude" value="{geo:lon}" title="Inventory with longitude in decimal degrees, must be used together with lat and radius" minimum="0" minInclusive="-180.0" maxInclusive="180.0"/>    
        <params:Parameter name="radius" uiDisplay="Radius" value="{geo:radius}" title="Inventory with the search radius in meters, must be used together with lat and lon" minimum="0" minInclusive="10" maxInclusive="6000000"/>    
        <params:Parameter name="geometry" uiDisplay="Geometry" value="{geo:geometry}" title="Inventory with a spatial extent overlapping this geometry" minimum="0">      
            <atom:link rel="profile" href="http://www.opengis.net/wkt/LINESTRING" title="This service accepts WKT LineStrings"/>      
            <atom:link rel="profile" href="http://www.opengis.net/wkt/POINT" title="This service accepts WKT Points"/>      
            <atom:link rel="profile" href="http://www.opengis.net/wkt/POLYGON" title="This service accepts WKT Polygons"/>      
        </params:Parameter>    
        <params:Parameter name="placeName" uiDisplay="Place name" value="{geo:name}" title="Inventory with a spatial location described by this name" minimum="0"/>    
        <params:Parameter name="startTime" uiDisplay="Start time" value="{time:start}" title="Inventory with a temporal extent containing this start time" minimum="0"/>    
        <params:Parameter name="endTime" uiDisplay="End time" value="{time:end}" title="Inventory with a temporal extent containing this end time" minimum="0"/>    
        <params:Parameter name="cursor" uiDisplay="Start page" value="{os:startPage}" title="Start page for the search result" minimum="0"/>    
        <params:Parameter name="numberOfResults" uiDisplay="Number of results" value="{os:count}" title="Maximum number of records in the search result" minimum="0" maxInclusive="2000"/>    
        <params:Parameter name="offset" uiDisplay="Start index" value="{os:startIndex}" title="0-based offset used to skip the specified number of results in the search result set" minimum="0"/>    
        <params:Parameter name="uid" uiDisplay="Unique identifier identifier" value="{geo:uid}" title="Inventory associated with this unique ID" minimum="0"/>    
        <params:Parameter name="parentIdentifier" uiDisplay="CMR Collection concept identifier" value="{eo:parentIdentifier}" title="Inventory associated with a dataset expressed as a CMR concept ID" minimum="0"/>    
        <params:Parameter name="clientId" uiDisplay="Client identifier" value="{referrer:source}" title="Client identifier to be used for metrics" minimum="0"/>    
    </os:Url>  

    <os:Query role="example" echo:shortName="MOD02QKM" echo:versionId="005" echo:dataCenter="LAADS" geo:box="-180.0,-90.0,180.0,90.0" time:start="2002-05-04T00:00:00Z" time:end="2009-05-04T00:00:00Z" title="Sample search"/>  

    <os:Attribution>NASA CMR</os:Attribution>  
    <os:SyndicationRight>open</os:SyndicationRight>  
    <os:Tags>CMR NASA CWIC CEOS-OS-BP-V1.1/L3 ESIP OGC granule pageOffset=1 indexOffset=0</os:Tags>  
</os:OpenSearchDescription>
```


## Retrieve Granules via OpenSearch

After retrieving the OSDD endpoint by querying through the IDN OpenSearch, OpenSearch clients
will sequentially interact with the host remote server for inventory search. The following table
shows the basic information about the use case of interacting with WGISS Connected Data Assets
via OpenSearch.



| **Use Case Overview**  |  | 
| -------- | --------- | 
| Title | Interact with CWIC OpenSearch APIs for inventory search | 
| Description | This use case describes steps for executing inventory search based on the IDN collection 
OSDD of interest. | 
| Actors | OpenSearch client, Collection-specific granule OpenSearch API | 
| Initial Status and Preconditions | Clients have the CWIC OpenSearch OSDD URL from IDN. | 

**Step 4**  
>  Obtain the collection specific OSDD from the IDN response.

In [37]:
url_osdd

'https://cmr.earthdata.nasa.gov/opensearch/granules/descriptor_document.xml?shortName=GEDI_L4A_AGB_Density_V2_1_2056'

**Step 5**  
>  Based on the OSDD, formulate an OpenSearch request for granules belonging to that collection.

In [38]:
# Extract URL template for granule search
root = ElementTree.fromstring(response.text)

ns = {'os': 'http://a9.com/-/spec/opensearch/1.1/'}
granule_search_template = root.find('os:Url[@rel="results"][@type="application/atom+xml"]', ns).attrib['template']

# collection_template = collection_url_atom.attrib['template']
granule_search_template

'https://cmr.earthdata.nasa.gov/opensearch/granules.atom?datasetId={echo:datasetId?}&shortName=GEDI_L4A_AGB_Density_V2_1_2056&versionId={echo:versionId?}&dataCenter={echo:dataCenter?}&boundingBox={geo:box?}&lat={geo:lat?}&lon={geo:lon?}&radius={geo:radius?}&geometry={geo:geometry?}&placeName={geo:name?}&startTime={time:start?}&endTime={time:end?}&cursor={os:startPage?}&numberOfResults={os:count?}&offset={os:startIndex?}&uid={geo:uid?}&parentIdentifier={eo:parentIdentifier?}'

In [39]:
request_url = get_api_request(granule_search_template, {'time:start': '1972-09-26T00:00:00Z'} )
request_url

'https://cmr.earthdata.nasa.gov/opensearch/granules.atom?&shortName=GEDI_L4A_AGB_Density_V2_1_2056&startTime=1972-09-26T00:00:00Z'

**Step 6**  
>  Parse the CWIC OpenSearch response and extract the identifier for the granule of interest from `id` in the `entry` element.

In [40]:
response = requests.get( request_url )

# xmlstr = minidom.parseString(response.text).toprettyxml(indent='   ', newl='')
# md("```xml\n" + xmlstr + "\n```\n")

root = ElementTree.fromstring(response.text)
# Extract <entry> element 
# el = root.find('{http://www.w3.org/2005/Atom}entry')
xmltxt = ElementTree.tostring(root, encoding='unicode', method='xml')
md("```xml\n" + xmltxt + "\n```\n")

```xml
<ns0:feed xmlns:dc="http://purl.org/dc/elements/1.1/" xmlns:ns0="http://www.w3.org/2005/Atom" xmlns:ns1="http://commons.esipfed.org/ns/discovery/1.2/" xmlns:ns2="http://a9.com/-/spec/opensearch/1.1/" xmlns:ns3="https://cmr.earthdata.nasa.gov/search/site/docs/search/api.html#atom" xmlns:ns4="http://a9.com/-/opensearch/extensions/time/1.0/" xmlns:ns6="http://www.georss.org/georss" ns1:version="1.2">
  <ns0:updated>2025-07-08T14:56:10.878Z</ns0:updated>
  <ns0:id>https://cmr.earthdata.nasa.gov/opensearch/granules.atom</ns0:id>
  <ns0:author>
    <ns0:name>CMR</ns0:name>
    <ns0:email>echodev@echo.nasa.gov</ns0:email>
  </ns0:author>
  <ns0:title type="text">ECHO granule metadata</ns0:title>
  <ns0:subtitle type="text">Search parameters: shortName =&gt; GEDI_L4A_AGB_Density_V2_1_2056 startTime =&gt; 1972-09-26T00:00:00Z</ns0:subtitle>
  <ns0:link href="https://cmr.earthdata.nasa.gov/opensearch/granules.atom?datasetId=GEDI L4A Footprint Level Aboveground Biomass Density, Version 2.1" hreflang="en-US" type="application/atom+xml" rel="up" />
  <ns0:link href="https://cmr.earthdata.nasa.gov/opensearch/granules.atom?&amp;shortName=GEDI_L4A_AGB_Density_V2_1_2056&amp;startTime=1972-09-26T00:00:00Z" hreflang="en-US" type="application/atom+xml" rel="self" />
  <ns0:link href="https://cmr.earthdata.nasa.gov/opensearch/granules.atom?&amp;shortName=GEDI_L4A_AGB_Density_V2_1_2056&amp;startTime=1972-09-26T00:00:00Z&amp;cursor=8507" hreflang="en-US" type="application/atom+xml" rel="last" />
  <ns0:link href="https://cmr.earthdata.nasa.gov/opensearch/granules.atom?&amp;shortName=GEDI_L4A_AGB_Density_V2_1_2056&amp;startTime=1972-09-26T00:00:00Z&amp;cursor=2" hreflang="en-US" type="application/atom+xml" rel="next" />
  <ns0:link href="https://cmr.earthdata.nasa.gov/opensearch/granules.atom?&amp;shortName=GEDI_L4A_AGB_Density_V2_1_2056&amp;startTime=1972-09-26T00:00:00Z&amp;cursor=1" hreflang="en-US" type="application/atom+xml" rel="first" />
  <ns0:link href="https://wiki.earthdata.nasa.gov/display/echo/Open+Search+API+release+information" hreflang="en-US" type="text/html" rel="describedBy" title="Release Notes" />
  <ns2:Query role="request" ns3:shortName="GEDI_L4A_AGB_Density_V2_1_2056" ns4:start="1972-09-26T00:00:00Z" />
  <ns2:totalResults>85064</ns2:totalResults>
  <ns2:itemsPerPage>10</ns2:itemsPerPage>
  <ns2:startIndex>1</ns2:startIndex>
  <ns0:entry>
    <ns0:id>https://cmr.earthdata.nasa.gov/opensearch/granules.atom?uid=G2613731222-ORNL_CLOUD</ns0:id>
    <ns0:title type="text">GEDI_L4A_AGB_Density_V2_1.GEDI04_A_2019107224731_O01958_01_T02638_02_002_02_V002.h5</ns0:title>
    <ns0:updated>2025-03-03T19:38:07.000Z</ns0:updated>
    <ns0:link rel="enclosure" title="Download GEDI04_A_2019107224731_O01958_01_T02638_02_002_02_V002.h5" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/protected/gedi/GEDI_L4A_AGB_Density_V2_1/data/GEDI04_A_2019107224731_O01958_01_T02638_02_002_02_V002.h5" />
    <ns0:link rel="http://esipfed.org/ns/fedsearch/1.1/s3#" title="This link provides direct download access via S3 to the granule" hreflang="en-US" href="s3://ornl-cumulus-prod-protected/gedi/GEDI_L4A_AGB_Density_V2_1/data/GEDI04_A_2019107224731_O01958_01_T02638_02_002_02_V002.h5" />
    <ns0:link rel="describedBy" title="ORNL DAAC Data Set Documentation (USER'S GUIDE)" hreflang="en-US" href="https://daac.ornl.gov/GEDI/guides/GEDI_L4A_AGB_Density_V2_1.html" type="text/html" />
    <ns0:link rel="via" title="Data set Landing Page DOI URL (DATA SET LANDING PAGE)" hreflang="en-US" href="https://doi.org/10.3334/ORNLDAAC/2056" />
    <ns0:link rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_ATBD_L4A_v1.0.pdf" type="application/pdf" />
    <ns0:link rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_L4A_AGB_Density_V2_1.pdf" type="application/pdf" />
    <ns0:link rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_L4A_V2_Common_Queries.pdf" type="application/pdf" />
    <ns0:link rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_L4A_V2_Product_Data_Dictionary.pdf" type="application/pdf" />
    <ns0:link rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_deleted_files_mw_163_182.txt" />
    <ns0:link rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_deleted_files_orbits_17591_09899.txt" />
    <ns0:link rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_revised_files_mw_163_182.txt" />
    <ns0:link rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_revised_files_orbits_17591_09899.txt" />
    <ns0:link rel="via" title="api endpoint to retrieve temporary credentials valid for same-region direct s3 access (VIEW RELATED INFORMATION)" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/s3credentials" />
    <ns0:link rel="http://esipfed.org/ns/fedsearch/1.1/service#" title="OPeNDAP request URL (GET DATA : OPENDAP DATA)" hreflang="en-US" href="https://opendap.earthdata.nasa.gov/collections/C2237824918-ORNL_CLOUD/granules/GEDI_L4A_AGB_Density_V2_1.GEDI04_A_2019107224731_O01958_01_T02638_02_002_02_V002.h5" />
    <ns0:link ns3:inherited="true" rel="describedBy" hreflang="en-US" href="https://daac.ornl.gov/GEDI/guides/GEDI_L4A_AGB_Density_V2_1.html" type="text/html">
      <ns3:inherited />
    </ns0:link>
    <ns0:link ns3:inherited="true" rel="via" hreflang="en-US" href="https://doi.org/10.3334/ORNLDAAC/2056">
      <ns3:inherited />
    </ns0:link>
    <ns0:link ns3:inherited="true" rel="http://esipfed.org/ns/fedsearch/1.1/service#" hreflang="en-US" href="https://opendap.earthdata.nasa.gov/">
      <ns3:inherited />
    </ns0:link>
    <ns0:link ns3:inherited="true" rel="via" hreflang="en-US" href="https://gedi.umd.edu">
      <ns3:inherited />
    </ns0:link>
    <ns0:link ns3:inherited="true" rel="describedBy" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_ATBD_L4A_v1.0.pdf" type="application/pdf">
      <ns3:inherited />
    </ns0:link>
    <ns0:link ns3:inherited="true" rel="describedBy" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_L4A_V2_Common_Queries.pdf" type="application/pdf">
      <ns3:inherited />
    </ns0:link>
    <ns0:link ns3:inherited="true" rel="describedBy" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_L4A_AGB_Density_V2_1.pdf" type="application/pdf">
      <ns3:inherited />
    </ns0:link>
    <ns0:link ns3:inherited="true" rel="describedBy" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_deleted_files_mw_163_182.txt">
      <ns3:inherited />
    </ns0:link>
    <ns0:link ns3:inherited="true" rel="describedBy" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_revised_files_mw_163_182.txt">
      <ns3:inherited />
    </ns0:link>
    <ns0:link ns3:inherited="true" rel="describedBy" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_deleted_files_orbits_17591_09899.txt">
      <ns3:inherited />
    </ns0:link>
    <ns0:link ns3:inherited="true" rel="describedBy" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_L4A_V2_Product_Data_Dictionary.pdf" type="application/pdf">
      <ns3:inherited />
    </ns0:link>
    <ns0:link ns3:inherited="true" rel="describedBy" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_revised_files_orbits_17591_09899.txt">
      <ns3:inherited />
    </ns0:link>
    <ns0:link href="https://cmr.earthdata.nasa.gov/search/concepts/G2613731222-ORNL_CLOUD.xml" hreflang="en-US" type="application/xml" rel="via" title="Product metadata" />
    <dc:identifier>G2613731222-ORNL_CLOUD</dc:identifier>
    <dc:date>2019-04-17T23:03:08.000Z/2019-04-17T23:10:25.000Z</dc:date>
    <ns3:datasetId>GEDI L4A Footprint Level Aboveground Biomass Density, Version 2.1</ns3:datasetId>
    <ns3:collectionConceptId>C2237824918-ORNL_CLOUD</ns3:collectionConceptId>
    <ns3:granuleSizeMB>86.071746</ns3:granuleSizeMB>
    <ns3:originalFormat>ECHO10</ns3:originalFormat>
    <ns3:dataCenter>ORNL_CLOUD</ns3:dataCenter>
    <ns3:coordinateSystem>GEODETIC</ns3:coordinateSystem>
    <ns6:polygon>-21.8339844 90.6664734 -18.9093285 93.1515121 -15.9455862 95.5423431 -12.9488249 97.8571167 -9.9263477 100.111557 -6.8845229 102.3207092 -3.8295021 104.4995575 -0.767069 106.6615677 0.096466 107.2697449 0.125464 107.228569 -0.738065 106.6203918 -3.800411 104.4583588 -6.8552461 102.2794342 -9.8967772 100.0701675 -12.9188538 97.8155899 -15.9151096 95.5006485 -18.8782387 93.109642 -21.8022995 90.6243057 -21.8339844 90.6664734</ns6:polygon>
  </ns0:entry>
  <ns0:entry>
    <ns0:id>https://cmr.earthdata.nasa.gov/opensearch/granules.atom?uid=G2613707402-ORNL_CLOUD</ns0:id>
    <ns0:title type="text">GEDI_L4A_AGB_Density_V2_1.GEDI04_A_2019107224731_O01958_02_T02638_02_002_02_V002.h5</ns0:title>
    <ns0:updated>2025-03-03T19:38:07.000Z</ns0:updated>
    <ns0:link rel="enclosure" title="Download GEDI04_A_2019107224731_O01958_02_T02638_02_002_02_V002.h5" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/protected/gedi/GEDI_L4A_AGB_Density_V2_1/data/GEDI04_A_2019107224731_O01958_02_T02638_02_002_02_V002.h5" />
    <ns0:link rel="http://esipfed.org/ns/fedsearch/1.1/s3#" title="This link provides direct download access via S3 to the granule" hreflang="en-US" href="s3://ornl-cumulus-prod-protected/gedi/GEDI_L4A_AGB_Density_V2_1/data/GEDI04_A_2019107224731_O01958_02_T02638_02_002_02_V002.h5" />
    <ns0:link rel="describedBy" title="ORNL DAAC Data Set Documentation (USER'S GUIDE)" hreflang="en-US" href="https://daac.ornl.gov/GEDI/guides/GEDI_L4A_AGB_Density_V2_1.html" type="text/html" />
    <ns0:link rel="via" title="Data set Landing Page DOI URL (DATA SET LANDING PAGE)" hreflang="en-US" href="https://doi.org/10.3334/ORNLDAAC/2056" />
    <ns0:link rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_ATBD_L4A_v1.0.pdf" type="application/pdf" />
    <ns0:link rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_L4A_AGB_Density_V2_1.pdf" type="application/pdf" />
    <ns0:link rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_L4A_V2_Common_Queries.pdf" type="application/pdf" />
    <ns0:link rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_L4A_V2_Product_Data_Dictionary.pdf" type="application/pdf" />
    <ns0:link rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_deleted_files_mw_163_182.txt" />
    <ns0:link rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_deleted_files_orbits_17591_09899.txt" />
    <ns0:link rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_revised_files_mw_163_182.txt" />
    <ns0:link rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_revised_files_orbits_17591_09899.txt" />
    <ns0:link rel="via" title="api endpoint to retrieve temporary credentials valid for same-region direct s3 access (VIEW RELATED INFORMATION)" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/s3credentials" />
    <ns0:link rel="http://esipfed.org/ns/fedsearch/1.1/service#" title="OPeNDAP request URL (GET DATA : OPENDAP DATA)" hreflang="en-US" href="https://opendap.earthdata.nasa.gov/collections/C2237824918-ORNL_CLOUD/granules/GEDI_L4A_AGB_Density_V2_1.GEDI04_A_2019107224731_O01958_02_T02638_02_002_02_V002.h5" />
    <ns0:link ns3:inherited="true" rel="describedBy" hreflang="en-US" href="https://daac.ornl.gov/GEDI/guides/GEDI_L4A_AGB_Density_V2_1.html" type="text/html">
      <ns3:inherited />
    </ns0:link>
    <ns0:link ns3:inherited="true" rel="via" hreflang="en-US" href="https://doi.org/10.3334/ORNLDAAC/2056">
      <ns3:inherited />
    </ns0:link>
    <ns0:link ns3:inherited="true" rel="http://esipfed.org/ns/fedsearch/1.1/service#" hreflang="en-US" href="https://opendap.earthdata.nasa.gov/">
      <ns3:inherited />
    </ns0:link>
    <ns0:link ns3:inherited="true" rel="via" hreflang="en-US" href="https://gedi.umd.edu">
      <ns3:inherited />
    </ns0:link>
    <ns0:link ns3:inherited="true" rel="describedBy" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_ATBD_L4A_v1.0.pdf" type="application/pdf">
      <ns3:inherited />
    </ns0:link>
    <ns0:link ns3:inherited="true" rel="describedBy" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_L4A_V2_Common_Queries.pdf" type="application/pdf">
      <ns3:inherited />
    </ns0:link>
    <ns0:link ns3:inherited="true" rel="describedBy" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_L4A_AGB_Density_V2_1.pdf" type="application/pdf">
      <ns3:inherited />
    </ns0:link>
    <ns0:link ns3:inherited="true" rel="describedBy" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_deleted_files_mw_163_182.txt">
      <ns3:inherited />
    </ns0:link>
    <ns0:link ns3:inherited="true" rel="describedBy" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_revised_files_mw_163_182.txt">
      <ns3:inherited />
    </ns0:link>
    <ns0:link ns3:inherited="true" rel="describedBy" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_deleted_files_orbits_17591_09899.txt">
      <ns3:inherited />
    </ns0:link>
    <ns0:link ns3:inherited="true" rel="describedBy" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_L4A_V2_Product_Data_Dictionary.pdf" type="application/pdf">
      <ns3:inherited />
    </ns0:link>
    <ns0:link ns3:inherited="true" rel="describedBy" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_revised_files_orbits_17591_09899.txt">
      <ns3:inherited />
    </ns0:link>
    <ns0:link href="https://cmr.earthdata.nasa.gov/search/concepts/G2613707402-ORNL_CLOUD.xml" hreflang="en-US" type="application/xml" rel="via" title="Product metadata" />
    <dc:identifier>G2613707402-ORNL_CLOUD</dc:identifier>
    <dc:date>2019-04-17T23:10:25.000Z/2019-04-17T23:33:09.000Z</dc:date>
    <ns3:datasetId>GEDI L4A Footprint Level Aboveground Biomass Density, Version 2.1</ns3:datasetId>
    <ns3:collectionConceptId>C2237824918-ORNL_CLOUD</ns3:collectionConceptId>
    <ns3:granuleSizeMB>233.259346</ns3:granuleSizeMB>
    <ns3:originalFormat>ECHO10</ns3:originalFormat>
    <ns3:dataCenter>ORNL_CLOUD</ns3:dataCenter>
    <ns3:coordinateSystem>GEODETIC</ns3:coordinateSystem>
    <ns6:polygon>0.096889 107.2700424 3.1605549 109.4303436 6.2186122 111.6051331 9.2651892 113.8084488 12.2944918 116.0545349 15.3001385 118.3584747 18.2749271 120.7357635 21.2112904 123.2036057 24.1017456 125.7802048 26.9362698 128.4854889 29.7046547 131.3400726 32.3950768 134.3668213 34.9940529 137.589798 37.4862328 141.0342407 43.9677429 152.5696564 45.8524971 157.1061401 47.5218086 161.9548035 48.9514313 167.1147919 50.1127739 172.559967 50.9826126 178.2515564 51.5400848 -175.8731384 51.7517281 -171.0981903 51.8020554 -171.1013336 51.5901413 -175.8820496 51.0319939 178.2358551 50.1611557 172.5381622 48.998558 167.0877228 47.5674973 161.9234161 45.8966331 157.0713043 44.0101318 152.5318604 37.5243111 140.9927063 35.0307579 137.547699 32.4305153 134.3244476 29.7389412 131.2976074 26.9695206 128.4430542 24.1340752 125.7378922 21.2428226 123.1614761 18.3057709 120.6938324 15.3304052 118.3167343 12.3242941 116.0129776 9.2946329 113.7670441 6.2478008 111.5638504 3.1895959 109.3891296 0.125886 107.2288666 0.096889 107.2700424</ns6:polygon>
  </ns0:entry>
  <ns0:entry>
    <ns0:id>https://cmr.earthdata.nasa.gov/opensearch/granules.atom?uid=G2613709653-ORNL_CLOUD</ns0:id>
    <ns0:title type="text">GEDI_L4A_AGB_Density_V2_1.GEDI04_A_2019107224731_O01958_03_T02638_02_002_02_V002.h5</ns0:title>
    <ns0:updated>2025-03-03T19:38:07.000Z</ns0:updated>
    <ns0:link rel="enclosure" title="Download GEDI04_A_2019107224731_O01958_03_T02638_02_002_02_V002.h5" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/protected/gedi/GEDI_L4A_AGB_Density_V2_1/data/GEDI04_A_2019107224731_O01958_03_T02638_02_002_02_V002.h5" />
    <ns0:link rel="http://esipfed.org/ns/fedsearch/1.1/s3#" title="This link provides direct download access via S3 to the granule" hreflang="en-US" href="s3://ornl-cumulus-prod-protected/gedi/GEDI_L4A_AGB_Density_V2_1/data/GEDI04_A_2019107224731_O01958_03_T02638_02_002_02_V002.h5" />
    <ns0:link rel="describedBy" title="ORNL DAAC Data Set Documentation (USER'S GUIDE)" hreflang="en-US" href="https://daac.ornl.gov/GEDI/guides/GEDI_L4A_AGB_Density_V2_1.html" type="text/html" />
    <ns0:link rel="via" title="Data set Landing Page DOI URL (DATA SET LANDING PAGE)" hreflang="en-US" href="https://doi.org/10.3334/ORNLDAAC/2056" />
    <ns0:link rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_ATBD_L4A_v1.0.pdf" type="application/pdf" />
    <ns0:link rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_L4A_AGB_Density_V2_1.pdf" type="application/pdf" />
    <ns0:link rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_L4A_V2_Common_Queries.pdf" type="application/pdf" />
    <ns0:link rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_L4A_V2_Product_Data_Dictionary.pdf" type="application/pdf" />
    <ns0:link rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_deleted_files_mw_163_182.txt" />
    <ns0:link rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_deleted_files_orbits_17591_09899.txt" />
    <ns0:link rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_revised_files_mw_163_182.txt" />
    <ns0:link rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_revised_files_orbits_17591_09899.txt" />
    <ns0:link rel="via" title="api endpoint to retrieve temporary credentials valid for same-region direct s3 access (VIEW RELATED INFORMATION)" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/s3credentials" />
    <ns0:link rel="http://esipfed.org/ns/fedsearch/1.1/service#" title="OPeNDAP request URL (GET DATA : OPENDAP DATA)" hreflang="en-US" href="https://opendap.earthdata.nasa.gov/collections/C2237824918-ORNL_CLOUD/granules/GEDI_L4A_AGB_Density_V2_1.GEDI04_A_2019107224731_O01958_03_T02638_02_002_02_V002.h5" />
    <ns0:link ns3:inherited="true" rel="describedBy" hreflang="en-US" href="https://daac.ornl.gov/GEDI/guides/GEDI_L4A_AGB_Density_V2_1.html" type="text/html">
      <ns3:inherited />
    </ns0:link>
    <ns0:link ns3:inherited="true" rel="via" hreflang="en-US" href="https://doi.org/10.3334/ORNLDAAC/2056">
      <ns3:inherited />
    </ns0:link>
    <ns0:link ns3:inherited="true" rel="http://esipfed.org/ns/fedsearch/1.1/service#" hreflang="en-US" href="https://opendap.earthdata.nasa.gov/">
      <ns3:inherited />
    </ns0:link>
    <ns0:link ns3:inherited="true" rel="via" hreflang="en-US" href="https://gedi.umd.edu">
      <ns3:inherited />
    </ns0:link>
    <ns0:link ns3:inherited="true" rel="describedBy" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_ATBD_L4A_v1.0.pdf" type="application/pdf">
      <ns3:inherited />
    </ns0:link>
    <ns0:link ns3:inherited="true" rel="describedBy" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_L4A_V2_Common_Queries.pdf" type="application/pdf">
      <ns3:inherited />
    </ns0:link>
    <ns0:link ns3:inherited="true" rel="describedBy" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_L4A_AGB_Density_V2_1.pdf" type="application/pdf">
      <ns3:inherited />
    </ns0:link>
    <ns0:link ns3:inherited="true" rel="describedBy" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_deleted_files_mw_163_182.txt">
      <ns3:inherited />
    </ns0:link>
    <ns0:link ns3:inherited="true" rel="describedBy" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_revised_files_mw_163_182.txt">
      <ns3:inherited />
    </ns0:link>
    <ns0:link ns3:inherited="true" rel="describedBy" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_deleted_files_orbits_17591_09899.txt">
      <ns3:inherited />
    </ns0:link>
    <ns0:link ns3:inherited="true" rel="describedBy" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_L4A_V2_Product_Data_Dictionary.pdf" type="application/pdf">
      <ns3:inherited />
    </ns0:link>
    <ns0:link ns3:inherited="true" rel="describedBy" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_revised_files_orbits_17591_09899.txt">
      <ns3:inherited />
    </ns0:link>
    <ns0:link href="https://cmr.earthdata.nasa.gov/search/concepts/G2613709653-ORNL_CLOUD.xml" hreflang="en-US" type="application/xml" rel="via" title="Product metadata" />
    <dc:identifier>G2613709653-ORNL_CLOUD</dc:identifier>
    <dc:date>2019-04-17T23:36:31.000Z/2019-04-17T23:56:45.000Z</dc:date>
    <ns3:datasetId>GEDI L4A Footprint Level Aboveground Biomass Density, Version 2.1</ns3:datasetId>
    <ns3:collectionConceptId>C2237824918-ORNL_CLOUD</ns3:collectionConceptId>
    <ns3:granuleSizeMB>289.630746</ns3:granuleSizeMB>
    <ns3:originalFormat>ECHO10</ns3:originalFormat>
    <ns3:dataCenter>ORNL_CLOUD</ns3:dataCenter>
    <ns3:coordinateSystem>GEODETIC</ns3:coordinateSystem>
    <ns6:polygon>50.3259239 -151.1632233 49.2252464 -145.6598206 47.8514786 -140.4356079 46.2308769 -135.5182343 44.3904305 -130.9140472 42.3554993 -126.6189728 40.1509552 -122.6179504 37.7999954 -118.8920975 35.3218842 -115.4155655 32.7349548 -112.1641464 30.0544357 -109.1125031 27.2942162 -106.2359772 24.4663277 -103.5119019 21.5813046 -100.9187241 18.6484966 -98.4365997 15.6760359 -96.0472336 12.6721449 -93.7337112 9.6435022 -91.4798203 6.596983 -89.270813 3.5376461 -87.0910721 0.472189 -84.9280548 -0.233585 -84.43116 -0.204593 -84.3899841 0.501189 -84.886879 3.5667281 -87.049881 6.6262422 -89.2295456 9.6730461 -91.438446 12.7020788 -93.6921921 15.7064657 -96.0055466 18.6795368 -98.3947372 21.6130619 -100.8766861 24.4989128 -103.4697113 27.3277473 -106.1936874 30.0890274 -109.0702133 32.7707253 -112.121994 35.3589401 -115.3737564 37.8384399 -118.850914 40.1908798 -122.577774 42.3969688 -126.5802917 44.4334869 -130.8774719 46.2755051 -135.4844971 47.8976212 -140.4055328 49.2727737 -145.6343231 50.3745461 -151.1426544 50.3259239 -151.1632233</ns6:polygon>
  </ns0:entry>
  <ns0:entry>
    <ns0:id>https://cmr.earthdata.nasa.gov/opensearch/granules.atom?uid=G2613767029-ORNL_CLOUD</ns0:id>
    <ns0:title type="text">GEDI_L4A_AGB_Density_V2_1.GEDI04_A_2019107224731_O01958_04_T02638_02_002_02_V002.h5</ns0:title>
    <ns0:updated>2025-03-03T19:38:07.000Z</ns0:updated>
    <ns0:link rel="enclosure" title="Download GEDI04_A_2019107224731_O01958_04_T02638_02_002_02_V002.h5" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/protected/gedi/GEDI_L4A_AGB_Density_V2_1/data/GEDI04_A_2019107224731_O01958_04_T02638_02_002_02_V002.h5" />
    <ns0:link rel="http://esipfed.org/ns/fedsearch/1.1/s3#" title="This link provides direct download access via S3 to the granule" hreflang="en-US" href="s3://ornl-cumulus-prod-protected/gedi/GEDI_L4A_AGB_Density_V2_1/data/GEDI04_A_2019107224731_O01958_04_T02638_02_002_02_V002.h5" />
    <ns0:link rel="describedBy" title="ORNL DAAC Data Set Documentation (USER'S GUIDE)" hreflang="en-US" href="https://daac.ornl.gov/GEDI/guides/GEDI_L4A_AGB_Density_V2_1.html" type="text/html" />
    <ns0:link rel="via" title="Data set Landing Page DOI URL (DATA SET LANDING PAGE)" hreflang="en-US" href="https://doi.org/10.3334/ORNLDAAC/2056" />
    <ns0:link rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_ATBD_L4A_v1.0.pdf" type="application/pdf" />
    <ns0:link rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_L4A_AGB_Density_V2_1.pdf" type="application/pdf" />
    <ns0:link rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_L4A_V2_Common_Queries.pdf" type="application/pdf" />
    <ns0:link rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_L4A_V2_Product_Data_Dictionary.pdf" type="application/pdf" />
    <ns0:link rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_deleted_files_mw_163_182.txt" />
    <ns0:link rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_deleted_files_orbits_17591_09899.txt" />
    <ns0:link rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_revised_files_mw_163_182.txt" />
    <ns0:link rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_revised_files_orbits_17591_09899.txt" />
    <ns0:link rel="via" title="api endpoint to retrieve temporary credentials valid for same-region direct s3 access (VIEW RELATED INFORMATION)" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/s3credentials" />
    <ns0:link rel="http://esipfed.org/ns/fedsearch/1.1/service#" title="OPeNDAP request URL (GET DATA : OPENDAP DATA)" hreflang="en-US" href="https://opendap.earthdata.nasa.gov/collections/C2237824918-ORNL_CLOUD/granules/GEDI_L4A_AGB_Density_V2_1.GEDI04_A_2019107224731_O01958_04_T02638_02_002_02_V002.h5" />
    <ns0:link ns3:inherited="true" rel="describedBy" hreflang="en-US" href="https://daac.ornl.gov/GEDI/guides/GEDI_L4A_AGB_Density_V2_1.html" type="text/html">
      <ns3:inherited />
    </ns0:link>
    <ns0:link ns3:inherited="true" rel="via" hreflang="en-US" href="https://doi.org/10.3334/ORNLDAAC/2056">
      <ns3:inherited />
    </ns0:link>
    <ns0:link ns3:inherited="true" rel="http://esipfed.org/ns/fedsearch/1.1/service#" hreflang="en-US" href="https://opendap.earthdata.nasa.gov/">
      <ns3:inherited />
    </ns0:link>
    <ns0:link ns3:inherited="true" rel="via" hreflang="en-US" href="https://gedi.umd.edu">
      <ns3:inherited />
    </ns0:link>
    <ns0:link ns3:inherited="true" rel="describedBy" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_ATBD_L4A_v1.0.pdf" type="application/pdf">
      <ns3:inherited />
    </ns0:link>
    <ns0:link ns3:inherited="true" rel="describedBy" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_L4A_V2_Common_Queries.pdf" type="application/pdf">
      <ns3:inherited />
    </ns0:link>
    <ns0:link ns3:inherited="true" rel="describedBy" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_L4A_AGB_Density_V2_1.pdf" type="application/pdf">
      <ns3:inherited />
    </ns0:link>
    <ns0:link ns3:inherited="true" rel="describedBy" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_deleted_files_mw_163_182.txt">
      <ns3:inherited />
    </ns0:link>
    <ns0:link ns3:inherited="true" rel="describedBy" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_revised_files_mw_163_182.txt">
      <ns3:inherited />
    </ns0:link>
    <ns0:link ns3:inherited="true" rel="describedBy" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_deleted_files_orbits_17591_09899.txt">
      <ns3:inherited />
    </ns0:link>
    <ns0:link ns3:inherited="true" rel="describedBy" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_L4A_V2_Product_Data_Dictionary.pdf" type="application/pdf">
      <ns3:inherited />
    </ns0:link>
    <ns0:link ns3:inherited="true" rel="describedBy" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_revised_files_orbits_17591_09899.txt">
      <ns3:inherited />
    </ns0:link>
    <ns0:link href="https://cmr.earthdata.nasa.gov/search/concepts/G2613767029-ORNL_CLOUD.xml" hreflang="en-US" type="application/xml" rel="via" title="Product metadata" />
    <dc:identifier>G2613767029-ORNL_CLOUD</dc:identifier>
    <dc:date>2019-04-17T23:56:45.000Z/2019-04-17T23:59:42.000Z</dc:date>
    <ns3:datasetId>GEDI L4A Footprint Level Aboveground Biomass Density, Version 2.1</ns3:datasetId>
    <ns3:collectionConceptId>C2237824918-ORNL_CLOUD</ns3:collectionConceptId>
    <ns3:granuleSizeMB>40.975817</ns3:granuleSizeMB>
    <ns3:originalFormat>ECHO10</ns3:originalFormat>
    <ns3:dataCenter>ORNL_CLOUD</ns3:dataCenter>
    <ns3:coordinateSystem>GEODETIC</ns3:coordinateSystem>
    <ns6:polygon>-0.23401 -84.4308624 -3.2991791 -82.268837 -6.3571992 -80.0932083 -9.2515869 -77.9995956 -9.2221441 -77.9581985 -6.328001 -80.051918 -3.2701299 -82.2276306 -0.20501 -84.3896866 -0.23401 -84.4308624</ns6:polygon>
  </ns0:entry>
  <ns0:entry>
    <ns0:id>https://cmr.earthdata.nasa.gov/opensearch/granules.atom?uid=G2613741178-ORNL_CLOUD</ns0:id>
    <ns0:title type="text">GEDI_L4A_AGB_Density_V2_1.GEDI04_A_2019108002012_O01959_01_T03909_02_002_02_V002.h5</ns0:title>
    <ns0:updated>2025-03-03T19:38:07.000Z</ns0:updated>
    <ns0:link rel="enclosure" title="Download GEDI04_A_2019108002012_O01959_01_T03909_02_002_02_V002.h5" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/protected/gedi/GEDI_L4A_AGB_Density_V2_1/data/GEDI04_A_2019108002012_O01959_01_T03909_02_002_02_V002.h5" />
    <ns0:link rel="http://esipfed.org/ns/fedsearch/1.1/s3#" title="This link provides direct download access via S3 to the granule" hreflang="en-US" href="s3://ornl-cumulus-prod-protected/gedi/GEDI_L4A_AGB_Density_V2_1/data/GEDI04_A_2019108002012_O01959_01_T03909_02_002_02_V002.h5" />
    <ns0:link rel="describedBy" title="ORNL DAAC Data Set Documentation (USER'S GUIDE)" hreflang="en-US" href="https://daac.ornl.gov/GEDI/guides/GEDI_L4A_AGB_Density_V2_1.html" type="text/html" />
    <ns0:link rel="via" title="Data set Landing Page DOI URL (DATA SET LANDING PAGE)" hreflang="en-US" href="https://doi.org/10.3334/ORNLDAAC/2056" />
    <ns0:link rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_ATBD_L4A_v1.0.pdf" type="application/pdf" />
    <ns0:link rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_L4A_AGB_Density_V2_1.pdf" type="application/pdf" />
    <ns0:link rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_L4A_V2_Common_Queries.pdf" type="application/pdf" />
    <ns0:link rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_L4A_V2_Product_Data_Dictionary.pdf" type="application/pdf" />
    <ns0:link rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_deleted_files_mw_163_182.txt" />
    <ns0:link rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_deleted_files_orbits_17591_09899.txt" />
    <ns0:link rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_revised_files_mw_163_182.txt" />
    <ns0:link rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_revised_files_orbits_17591_09899.txt" />
    <ns0:link rel="via" title="api endpoint to retrieve temporary credentials valid for same-region direct s3 access (VIEW RELATED INFORMATION)" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/s3credentials" />
    <ns0:link rel="http://esipfed.org/ns/fedsearch/1.1/service#" title="OPeNDAP request URL (GET DATA : OPENDAP DATA)" hreflang="en-US" href="https://opendap.earthdata.nasa.gov/collections/C2237824918-ORNL_CLOUD/granules/GEDI_L4A_AGB_Density_V2_1.GEDI04_A_2019108002012_O01959_01_T03909_02_002_02_V002.h5" />
    <ns0:link ns3:inherited="true" rel="describedBy" hreflang="en-US" href="https://daac.ornl.gov/GEDI/guides/GEDI_L4A_AGB_Density_V2_1.html" type="text/html">
      <ns3:inherited />
    </ns0:link>
    <ns0:link ns3:inherited="true" rel="via" hreflang="en-US" href="https://doi.org/10.3334/ORNLDAAC/2056">
      <ns3:inherited />
    </ns0:link>
    <ns0:link ns3:inherited="true" rel="http://esipfed.org/ns/fedsearch/1.1/service#" hreflang="en-US" href="https://opendap.earthdata.nasa.gov/">
      <ns3:inherited />
    </ns0:link>
    <ns0:link ns3:inherited="true" rel="via" hreflang="en-US" href="https://gedi.umd.edu">
      <ns3:inherited />
    </ns0:link>
    <ns0:link ns3:inherited="true" rel="describedBy" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_ATBD_L4A_v1.0.pdf" type="application/pdf">
      <ns3:inherited />
    </ns0:link>
    <ns0:link ns3:inherited="true" rel="describedBy" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_L4A_V2_Common_Queries.pdf" type="application/pdf">
      <ns3:inherited />
    </ns0:link>
    <ns0:link ns3:inherited="true" rel="describedBy" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_L4A_AGB_Density_V2_1.pdf" type="application/pdf">
      <ns3:inherited />
    </ns0:link>
    <ns0:link ns3:inherited="true" rel="describedBy" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_deleted_files_mw_163_182.txt">
      <ns3:inherited />
    </ns0:link>
    <ns0:link ns3:inherited="true" rel="describedBy" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_revised_files_mw_163_182.txt">
      <ns3:inherited />
    </ns0:link>
    <ns0:link ns3:inherited="true" rel="describedBy" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_deleted_files_orbits_17591_09899.txt">
      <ns3:inherited />
    </ns0:link>
    <ns0:link ns3:inherited="true" rel="describedBy" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_L4A_V2_Product_Data_Dictionary.pdf" type="application/pdf">
      <ns3:inherited />
    </ns0:link>
    <ns0:link ns3:inherited="true" rel="describedBy" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_revised_files_orbits_17591_09899.txt">
      <ns3:inherited />
    </ns0:link>
    <ns0:link href="https://cmr.earthdata.nasa.gov/search/concepts/G2613741178-ORNL_CLOUD.xml" hreflang="en-US" type="application/xml" rel="via" title="Product metadata" />
    <dc:identifier>G2613741178-ORNL_CLOUD</dc:identifier>
    <dc:date>2019-04-18T00:41:33.000Z/2019-04-18T00:43:06.000Z</dc:date>
    <ns3:datasetId>GEDI L4A Footprint Level Aboveground Biomass Density, Version 2.1</ns3:datasetId>
    <ns3:collectionConceptId>C2237824918-ORNL_CLOUD</ns3:collectionConceptId>
    <ns3:granuleSizeMB>17.02199</ns3:granuleSizeMB>
    <ns3:originalFormat>ECHO10</ns3:originalFormat>
    <ns3:dataCenter>ORNL_CLOUD</ns3:dataCenter>
    <ns3:coordinateSystem>GEODETIC</ns3:coordinateSystem>
    <ns6:polygon>-4.6459689 80.3302689 -1.585824 82.4953766 0.09879 83.6814117 0.127777 83.6402283 -1.556802 82.4542084 -4.6168618 80.289032 -4.6459689 80.3302689</ns6:polygon>
  </ns0:entry>
  <ns0:entry>
    <ns0:id>https://cmr.earthdata.nasa.gov/opensearch/granules.atom?uid=G2613728635-ORNL_CLOUD</ns0:id>
    <ns0:title type="text">GEDI_L4A_AGB_Density_V2_1.GEDI04_A_2019108002012_O01959_02_T03909_02_002_02_V002.h5</ns0:title>
    <ns0:updated>2025-03-03T19:38:07.000Z</ns0:updated>
    <ns0:link rel="enclosure" title="Download GEDI04_A_2019108002012_O01959_02_T03909_02_002_02_V002.h5" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/protected/gedi/GEDI_L4A_AGB_Density_V2_1/data/GEDI04_A_2019108002012_O01959_02_T03909_02_002_02_V002.h5" />
    <ns0:link rel="http://esipfed.org/ns/fedsearch/1.1/s3#" title="This link provides direct download access via S3 to the granule" hreflang="en-US" href="s3://ornl-cumulus-prod-protected/gedi/GEDI_L4A_AGB_Density_V2_1/data/GEDI04_A_2019108002012_O01959_02_T03909_02_002_02_V002.h5" />
    <ns0:link rel="describedBy" title="ORNL DAAC Data Set Documentation (USER'S GUIDE)" hreflang="en-US" href="https://daac.ornl.gov/GEDI/guides/GEDI_L4A_AGB_Density_V2_1.html" type="text/html" />
    <ns0:link rel="via" title="Data set Landing Page DOI URL (DATA SET LANDING PAGE)" hreflang="en-US" href="https://doi.org/10.3334/ORNLDAAC/2056" />
    <ns0:link rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_ATBD_L4A_v1.0.pdf" type="application/pdf" />
    <ns0:link rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_L4A_AGB_Density_V2_1.pdf" type="application/pdf" />
    <ns0:link rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_L4A_V2_Common_Queries.pdf" type="application/pdf" />
    <ns0:link rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_L4A_V2_Product_Data_Dictionary.pdf" type="application/pdf" />
    <ns0:link rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_deleted_files_mw_163_182.txt" />
    <ns0:link rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_deleted_files_orbits_17591_09899.txt" />
    <ns0:link rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_revised_files_mw_163_182.txt" />
    <ns0:link rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_revised_files_orbits_17591_09899.txt" />
    <ns0:link rel="via" title="api endpoint to retrieve temporary credentials valid for same-region direct s3 access (VIEW RELATED INFORMATION)" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/s3credentials" />
    <ns0:link rel="http://esipfed.org/ns/fedsearch/1.1/service#" title="OPeNDAP request URL (GET DATA : OPENDAP DATA)" hreflang="en-US" href="https://opendap.earthdata.nasa.gov/collections/C2237824918-ORNL_CLOUD/granules/GEDI_L4A_AGB_Density_V2_1.GEDI04_A_2019108002012_O01959_02_T03909_02_002_02_V002.h5" />
    <ns0:link ns3:inherited="true" rel="describedBy" hreflang="en-US" href="https://daac.ornl.gov/GEDI/guides/GEDI_L4A_AGB_Density_V2_1.html" type="text/html">
      <ns3:inherited />
    </ns0:link>
    <ns0:link ns3:inherited="true" rel="via" hreflang="en-US" href="https://doi.org/10.3334/ORNLDAAC/2056">
      <ns3:inherited />
    </ns0:link>
    <ns0:link ns3:inherited="true" rel="http://esipfed.org/ns/fedsearch/1.1/service#" hreflang="en-US" href="https://opendap.earthdata.nasa.gov/">
      <ns3:inherited />
    </ns0:link>
    <ns0:link ns3:inherited="true" rel="via" hreflang="en-US" href="https://gedi.umd.edu">
      <ns3:inherited />
    </ns0:link>
    <ns0:link ns3:inherited="true" rel="describedBy" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_ATBD_L4A_v1.0.pdf" type="application/pdf">
      <ns3:inherited />
    </ns0:link>
    <ns0:link ns3:inherited="true" rel="describedBy" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_L4A_V2_Common_Queries.pdf" type="application/pdf">
      <ns3:inherited />
    </ns0:link>
    <ns0:link ns3:inherited="true" rel="describedBy" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_L4A_AGB_Density_V2_1.pdf" type="application/pdf">
      <ns3:inherited />
    </ns0:link>
    <ns0:link ns3:inherited="true" rel="describedBy" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_deleted_files_mw_163_182.txt">
      <ns3:inherited />
    </ns0:link>
    <ns0:link ns3:inherited="true" rel="describedBy" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_revised_files_mw_163_182.txt">
      <ns3:inherited />
    </ns0:link>
    <ns0:link ns3:inherited="true" rel="describedBy" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_deleted_files_orbits_17591_09899.txt">
      <ns3:inherited />
    </ns0:link>
    <ns0:link ns3:inherited="true" rel="describedBy" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_L4A_V2_Product_Data_Dictionary.pdf" type="application/pdf">
      <ns3:inherited />
    </ns0:link>
    <ns0:link ns3:inherited="true" rel="describedBy" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_revised_files_orbits_17591_09899.txt">
      <ns3:inherited />
    </ns0:link>
    <ns0:link href="https://cmr.earthdata.nasa.gov/search/concepts/G2613728635-ORNL_CLOUD.xml" hreflang="en-US" type="application/xml" rel="via" title="Product metadata" />
    <dc:identifier>G2613728635-ORNL_CLOUD</dc:identifier>
    <dc:date>2019-04-18T00:43:06.000Z/2019-04-18T01:06:16.000Z</dc:date>
    <ns3:datasetId>GEDI L4A Footprint Level Aboveground Biomass Density, Version 2.1</ns3:datasetId>
    <ns3:collectionConceptId>C2237824918-ORNL_CLOUD</ns3:collectionConceptId>
    <ns3:granuleSizeMB>356.752358</ns3:granuleSizeMB>
    <ns3:originalFormat>ECHO10</ns3:originalFormat>
    <ns3:dataCenter>ORNL_CLOUD</ns3:dataCenter>
    <ns3:coordinateSystem>GEODETIC</ns3:coordinateSystem>
    <ns6:polygon>0.099208 83.6817093 3.1628549 85.8422318 6.2209601 88.0170898 9.2673845 90.2206497 12.2967377 92.4668427 15.3021154 94.7709198 18.2766266 97.1482544 21.2129631 99.6162949 24.1030331 102.1928482 26.9373379 104.8982239 29.7057495 107.7529068 32.3960228 110.7800674 34.9951553 114.0032806 37.4867973 117.4478683 39.8553276 121.1404724 42.0786362 125.1035538 44.1362953 129.3586578 46.0028419 133.9211731 47.6527596 138.797226 49.0604744 143.9837189 50.1977348 149.4519958 51.0410347 155.1615295 51.5710526 161.0489502 51.7738762 167.0356445 51.7756119 168.0159607 51.8259735 168.0162811 51.824234 167.0345459 51.6211472 161.0406342 51.0904884 155.1463776 50.2462158 149.4306793 49.1077156 143.957077 47.6985779 138.7661743 46.0471153 133.8866119 44.1789742 129.3213959 42.1197357 125.0643234 39.8948784 121.0998459 37.5248833 117.4063263 35.0318604 113.9611816 32.4314651 110.7376862 29.740036 107.7104263 26.9705925 104.8557968 24.1353664 102.1505432 21.2444973 99.574173 18.3074703 97.1063232 15.3323851 94.729187 12.3265409 92.425293 9.2968321 90.1792526 6.2501502 87.9758072 3.1918981 85.8010254 0.128207 83.6405334 0.099208 83.6817093</ns6:polygon>
  </ns0:entry>
  <ns0:entry>
    <ns0:id>https://cmr.earthdata.nasa.gov/opensearch/granules.atom?uid=G2613738123-ORNL_CLOUD</ns0:id>
    <ns0:title type="text">GEDI_L4A_AGB_Density_V2_1.GEDI04_A_2019108002012_O01959_03_T03909_02_002_02_V002.h5</ns0:title>
    <ns0:updated>2025-03-03T19:38:07.000Z</ns0:updated>
    <ns0:link rel="enclosure" title="Download GEDI04_A_2019108002012_O01959_03_T03909_02_002_02_V002.h5" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/protected/gedi/GEDI_L4A_AGB_Density_V2_1/data/GEDI04_A_2019108002012_O01959_03_T03909_02_002_02_V002.h5" />
    <ns0:link rel="http://esipfed.org/ns/fedsearch/1.1/s3#" title="This link provides direct download access via S3 to the granule" hreflang="en-US" href="s3://ornl-cumulus-prod-protected/gedi/GEDI_L4A_AGB_Density_V2_1/data/GEDI04_A_2019108002012_O01959_03_T03909_02_002_02_V002.h5" />
    <ns0:link rel="describedBy" title="ORNL DAAC Data Set Documentation (USER'S GUIDE)" hreflang="en-US" href="https://daac.ornl.gov/GEDI/guides/GEDI_L4A_AGB_Density_V2_1.html" type="text/html" />
    <ns0:link rel="via" title="Data set Landing Page DOI URL (DATA SET LANDING PAGE)" hreflang="en-US" href="https://doi.org/10.3334/ORNLDAAC/2056" />
    <ns0:link rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_ATBD_L4A_v1.0.pdf" type="application/pdf" />
    <ns0:link rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_L4A_AGB_Density_V2_1.pdf" type="application/pdf" />
    <ns0:link rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_L4A_V2_Common_Queries.pdf" type="application/pdf" />
    <ns0:link rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_L4A_V2_Product_Data_Dictionary.pdf" type="application/pdf" />
    <ns0:link rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_deleted_files_mw_163_182.txt" />
    <ns0:link rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_deleted_files_orbits_17591_09899.txt" />
    <ns0:link rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_revised_files_mw_163_182.txt" />
    <ns0:link rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_revised_files_orbits_17591_09899.txt" />
    <ns0:link rel="via" title="api endpoint to retrieve temporary credentials valid for same-region direct s3 access (VIEW RELATED INFORMATION)" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/s3credentials" />
    <ns0:link rel="http://esipfed.org/ns/fedsearch/1.1/service#" title="OPeNDAP request URL (GET DATA : OPENDAP DATA)" hreflang="en-US" href="https://opendap.earthdata.nasa.gov/collections/C2237824918-ORNL_CLOUD/granules/GEDI_L4A_AGB_Density_V2_1.GEDI04_A_2019108002012_O01959_03_T03909_02_002_02_V002.h5" />
    <ns0:link ns3:inherited="true" rel="describedBy" hreflang="en-US" href="https://daac.ornl.gov/GEDI/guides/GEDI_L4A_AGB_Density_V2_1.html" type="text/html">
      <ns3:inherited />
    </ns0:link>
    <ns0:link ns3:inherited="true" rel="via" hreflang="en-US" href="https://doi.org/10.3334/ORNLDAAC/2056">
      <ns3:inherited />
    </ns0:link>
    <ns0:link ns3:inherited="true" rel="http://esipfed.org/ns/fedsearch/1.1/service#" hreflang="en-US" href="https://opendap.earthdata.nasa.gov/">
      <ns3:inherited />
    </ns0:link>
    <ns0:link ns3:inherited="true" rel="via" hreflang="en-US" href="https://gedi.umd.edu">
      <ns3:inherited />
    </ns0:link>
    <ns0:link ns3:inherited="true" rel="describedBy" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_ATBD_L4A_v1.0.pdf" type="application/pdf">
      <ns3:inherited />
    </ns0:link>
    <ns0:link ns3:inherited="true" rel="describedBy" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_L4A_V2_Common_Queries.pdf" type="application/pdf">
      <ns3:inherited />
    </ns0:link>
    <ns0:link ns3:inherited="true" rel="describedBy" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_L4A_AGB_Density_V2_1.pdf" type="application/pdf">
      <ns3:inherited />
    </ns0:link>
    <ns0:link ns3:inherited="true" rel="describedBy" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_deleted_files_mw_163_182.txt">
      <ns3:inherited />
    </ns0:link>
    <ns0:link ns3:inherited="true" rel="describedBy" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_revised_files_mw_163_182.txt">
      <ns3:inherited />
    </ns0:link>
    <ns0:link ns3:inherited="true" rel="describedBy" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_deleted_files_orbits_17591_09899.txt">
      <ns3:inherited />
    </ns0:link>
    <ns0:link ns3:inherited="true" rel="describedBy" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_L4A_V2_Product_Data_Dictionary.pdf" type="application/pdf">
      <ns3:inherited />
    </ns0:link>
    <ns0:link ns3:inherited="true" rel="describedBy" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_revised_files_orbits_17591_09899.txt">
      <ns3:inherited />
    </ns0:link>
    <ns0:link href="https://cmr.earthdata.nasa.gov/search/concepts/G2613738123-ORNL_CLOUD.xml" hreflang="en-US" type="application/xml" rel="via" title="Product metadata" />
    <dc:identifier>G2613738123-ORNL_CLOUD</dc:identifier>
    <dc:date>2019-04-18T01:06:16.000Z/2019-04-18T01:08:30.000Z</dc:date>
    <ns3:datasetId>GEDI L4A Footprint Level Aboveground Biomass Density, Version 2.1</ns3:datasetId>
    <ns3:collectionConceptId>C2237824918-ORNL_CLOUD</ns3:collectionConceptId>
    <ns3:granuleSizeMB>27.728352</ns3:granuleSizeMB>
    <ns3:originalFormat>ECHO10</ns3:originalFormat>
    <ns3:dataCenter>ORNL_CLOUD</ns3:dataCenter>
    <ns3:coordinateSystem>GEODETIC</ns3:coordinateSystem>
    <ns6:polygon>51.7756081 168.0166779 51.5928535 174.006897 51.0830269 179.9024353 50.9306984 -178.8402405 50.9800682 -178.8244476 51.1325951 179.916626 51.643013 174.0141907 51.8259697 168.017334 51.7756081 168.0166779</ns6:polygon>
  </ns0:entry>
  <ns0:entry>
    <ns0:id>https://cmr.earthdata.nasa.gov/opensearch/granules.atom?uid=G2613712163-ORNL_CLOUD</ns0:id>
    <ns0:title type="text">GEDI_L4A_AGB_Density_V2_1.GEDI04_A_2019108002012_O01959_04_T03909_02_002_02_V002.h5</ns0:title>
    <ns0:updated>2025-03-03T19:38:07.000Z</ns0:updated>
    <ns0:link rel="enclosure" title="Download GEDI04_A_2019108002012_O01959_04_T03909_02_002_02_V002.h5" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/protected/gedi/GEDI_L4A_AGB_Density_V2_1/data/GEDI04_A_2019108002012_O01959_04_T03909_02_002_02_V002.h5" />
    <ns0:link rel="http://esipfed.org/ns/fedsearch/1.1/s3#" title="This link provides direct download access via S3 to the granule" hreflang="en-US" href="s3://ornl-cumulus-prod-protected/gedi/GEDI_L4A_AGB_Density_V2_1/data/GEDI04_A_2019108002012_O01959_04_T03909_02_002_02_V002.h5" />
    <ns0:link rel="describedBy" title="ORNL DAAC Data Set Documentation (USER'S GUIDE)" hreflang="en-US" href="https://daac.ornl.gov/GEDI/guides/GEDI_L4A_AGB_Density_V2_1.html" type="text/html" />
    <ns0:link rel="via" title="Data set Landing Page DOI URL (DATA SET LANDING PAGE)" hreflang="en-US" href="https://doi.org/10.3334/ORNLDAAC/2056" />
    <ns0:link rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_ATBD_L4A_v1.0.pdf" type="application/pdf" />
    <ns0:link rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_L4A_AGB_Density_V2_1.pdf" type="application/pdf" />
    <ns0:link rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_L4A_V2_Common_Queries.pdf" type="application/pdf" />
    <ns0:link rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_L4A_V2_Product_Data_Dictionary.pdf" type="application/pdf" />
    <ns0:link rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_deleted_files_mw_163_182.txt" />
    <ns0:link rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_deleted_files_orbits_17591_09899.txt" />
    <ns0:link rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_revised_files_mw_163_182.txt" />
    <ns0:link rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_revised_files_orbits_17591_09899.txt" />
    <ns0:link rel="via" title="api endpoint to retrieve temporary credentials valid for same-region direct s3 access (VIEW RELATED INFORMATION)" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/s3credentials" />
    <ns0:link rel="http://esipfed.org/ns/fedsearch/1.1/service#" title="OPeNDAP request URL (GET DATA : OPENDAP DATA)" hreflang="en-US" href="https://opendap.earthdata.nasa.gov/collections/C2237824918-ORNL_CLOUD/granules/GEDI_L4A_AGB_Density_V2_1.GEDI04_A_2019108002012_O01959_04_T03909_02_002_02_V002.h5" />
    <ns0:link ns3:inherited="true" rel="describedBy" hreflang="en-US" href="https://daac.ornl.gov/GEDI/guides/GEDI_L4A_AGB_Density_V2_1.html" type="text/html">
      <ns3:inherited />
    </ns0:link>
    <ns0:link ns3:inherited="true" rel="via" hreflang="en-US" href="https://doi.org/10.3334/ORNLDAAC/2056">
      <ns3:inherited />
    </ns0:link>
    <ns0:link ns3:inherited="true" rel="http://esipfed.org/ns/fedsearch/1.1/service#" hreflang="en-US" href="https://opendap.earthdata.nasa.gov/">
      <ns3:inherited />
    </ns0:link>
    <ns0:link ns3:inherited="true" rel="via" hreflang="en-US" href="https://gedi.umd.edu">
      <ns3:inherited />
    </ns0:link>
    <ns0:link ns3:inherited="true" rel="describedBy" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_ATBD_L4A_v1.0.pdf" type="application/pdf">
      <ns3:inherited />
    </ns0:link>
    <ns0:link ns3:inherited="true" rel="describedBy" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_L4A_V2_Common_Queries.pdf" type="application/pdf">
      <ns3:inherited />
    </ns0:link>
    <ns0:link ns3:inherited="true" rel="describedBy" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_L4A_AGB_Density_V2_1.pdf" type="application/pdf">
      <ns3:inherited />
    </ns0:link>
    <ns0:link ns3:inherited="true" rel="describedBy" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_deleted_files_mw_163_182.txt">
      <ns3:inherited />
    </ns0:link>
    <ns0:link ns3:inherited="true" rel="describedBy" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_revised_files_mw_163_182.txt">
      <ns3:inherited />
    </ns0:link>
    <ns0:link ns3:inherited="true" rel="describedBy" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_deleted_files_orbits_17591_09899.txt">
      <ns3:inherited />
    </ns0:link>
    <ns0:link ns3:inherited="true" rel="describedBy" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_L4A_V2_Product_Data_Dictionary.pdf" type="application/pdf">
      <ns3:inherited />
    </ns0:link>
    <ns0:link ns3:inherited="true" rel="describedBy" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_revised_files_orbits_17591_09899.txt">
      <ns3:inherited />
    </ns0:link>
    <ns0:link href="https://cmr.earthdata.nasa.gov/search/concepts/G2613712163-ORNL_CLOUD.xml" hreflang="en-US" type="application/xml" rel="via" title="Product metadata" />
    <dc:identifier>G2613712163-ORNL_CLOUD</dc:identifier>
    <dc:date>2019-04-18T01:37:02.000Z/2019-04-18T01:45:13.000Z</dc:date>
    <ns3:datasetId>GEDI L4A Footprint Level Aboveground Biomass Density, Version 2.1</ns3:datasetId>
    <ns3:collectionConceptId>C2237824918-ORNL_CLOUD</ns3:collectionConceptId>
    <ns3:granuleSizeMB>96.386522</ns3:granuleSizeMB>
    <ns3:originalFormat>ECHO10</ns3:originalFormat>
    <ns3:dataCenter>ORNL_CLOUD</ns3:dataCenter>
    <ns3:coordinateSystem>GEODETIC</ns3:coordinateSystem>
    <ns6:polygon>-23.0758801 -90.5495224 -25.9292355 -87.8976212 -28.719553 -85.1057281 -31.4343376 -82.1522675 -34.066246 -79.0105362 -36.5951424 -75.659523 -39.0072632 -72.0722198 -41.2829971 -68.2245407 -43.4022217 -64.0936508 -43.7361984 -63.3814812 -43.6937943 -63.3438873 -43.3601494 -64.0555573 -41.2424965 -68.1847229 -38.9682808 -72.0312042 -36.5575867 -75.6177444 -34.0300255 -78.9683075 -31.399332 -82.1098328 -28.6856613 -85.0632553 -25.8963356 -87.8552322 -23.0437202 -90.5074005 -23.0758801 -90.5495224</ns6:polygon>
  </ns0:entry>
  <ns0:entry>
    <ns0:id>https://cmr.earthdata.nasa.gov/opensearch/granules.atom?uid=G2613707648-ORNL_CLOUD</ns0:id>
    <ns0:title type="text">GEDI_L4A_AGB_Density_V2_1.GEDI04_A_2019108015253_O01960_01_T03910_02_002_02_V002.h5</ns0:title>
    <ns0:updated>2025-03-03T19:38:07.000Z</ns0:updated>
    <ns0:link rel="enclosure" title="Download GEDI04_A_2019108015253_O01960_01_T03910_02_002_02_V002.h5" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/protected/gedi/GEDI_L4A_AGB_Density_V2_1/data/GEDI04_A_2019108015253_O01960_01_T03910_02_002_02_V002.h5" />
    <ns0:link rel="http://esipfed.org/ns/fedsearch/1.1/s3#" title="This link provides direct download access via S3 to the granule" hreflang="en-US" href="s3://ornl-cumulus-prod-protected/gedi/GEDI_L4A_AGB_Density_V2_1/data/GEDI04_A_2019108015253_O01960_01_T03910_02_002_02_V002.h5" />
    <ns0:link rel="describedBy" title="ORNL DAAC Data Set Documentation (USER'S GUIDE)" hreflang="en-US" href="https://daac.ornl.gov/GEDI/guides/GEDI_L4A_AGB_Density_V2_1.html" type="text/html" />
    <ns0:link rel="via" title="Data set Landing Page DOI URL (DATA SET LANDING PAGE)" hreflang="en-US" href="https://doi.org/10.3334/ORNLDAAC/2056" />
    <ns0:link rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_ATBD_L4A_v1.0.pdf" type="application/pdf" />
    <ns0:link rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_L4A_AGB_Density_V2_1.pdf" type="application/pdf" />
    <ns0:link rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_L4A_V2_Common_Queries.pdf" type="application/pdf" />
    <ns0:link rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_L4A_V2_Product_Data_Dictionary.pdf" type="application/pdf" />
    <ns0:link rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_deleted_files_mw_163_182.txt" />
    <ns0:link rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_deleted_files_orbits_17591_09899.txt" />
    <ns0:link rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_revised_files_mw_163_182.txt" />
    <ns0:link rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_revised_files_orbits_17591_09899.txt" />
    <ns0:link rel="via" title="api endpoint to retrieve temporary credentials valid for same-region direct s3 access (VIEW RELATED INFORMATION)" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/s3credentials" />
    <ns0:link rel="http://esipfed.org/ns/fedsearch/1.1/service#" title="OPeNDAP request URL (GET DATA : OPENDAP DATA)" hreflang="en-US" href="https://opendap.earthdata.nasa.gov/collections/C2237824918-ORNL_CLOUD/granules/GEDI_L4A_AGB_Density_V2_1.GEDI04_A_2019108015253_O01960_01_T03910_02_002_02_V002.h5" />
    <ns0:link ns3:inherited="true" rel="describedBy" hreflang="en-US" href="https://daac.ornl.gov/GEDI/guides/GEDI_L4A_AGB_Density_V2_1.html" type="text/html">
      <ns3:inherited />
    </ns0:link>
    <ns0:link ns3:inherited="true" rel="via" hreflang="en-US" href="https://doi.org/10.3334/ORNLDAAC/2056">
      <ns3:inherited />
    </ns0:link>
    <ns0:link ns3:inherited="true" rel="http://esipfed.org/ns/fedsearch/1.1/service#" hreflang="en-US" href="https://opendap.earthdata.nasa.gov/">
      <ns3:inherited />
    </ns0:link>
    <ns0:link ns3:inherited="true" rel="via" hreflang="en-US" href="https://gedi.umd.edu">
      <ns3:inherited />
    </ns0:link>
    <ns0:link ns3:inherited="true" rel="describedBy" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_ATBD_L4A_v1.0.pdf" type="application/pdf">
      <ns3:inherited />
    </ns0:link>
    <ns0:link ns3:inherited="true" rel="describedBy" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_L4A_V2_Common_Queries.pdf" type="application/pdf">
      <ns3:inherited />
    </ns0:link>
    <ns0:link ns3:inherited="true" rel="describedBy" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_L4A_AGB_Density_V2_1.pdf" type="application/pdf">
      <ns3:inherited />
    </ns0:link>
    <ns0:link ns3:inherited="true" rel="describedBy" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_deleted_files_mw_163_182.txt">
      <ns3:inherited />
    </ns0:link>
    <ns0:link ns3:inherited="true" rel="describedBy" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_revised_files_mw_163_182.txt">
      <ns3:inherited />
    </ns0:link>
    <ns0:link ns3:inherited="true" rel="describedBy" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_deleted_files_orbits_17591_09899.txt">
      <ns3:inherited />
    </ns0:link>
    <ns0:link ns3:inherited="true" rel="describedBy" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_L4A_V2_Product_Data_Dictionary.pdf" type="application/pdf">
      <ns3:inherited />
    </ns0:link>
    <ns0:link ns3:inherited="true" rel="describedBy" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_revised_files_orbits_17591_09899.txt">
      <ns3:inherited />
    </ns0:link>
    <ns0:link href="https://cmr.earthdata.nasa.gov/search/concepts/G2613707648-ORNL_CLOUD.xml" hreflang="en-US" type="application/xml" rel="via" title="Product metadata" />
    <dc:identifier>G2613707648-ORNL_CLOUD</dc:identifier>
    <dc:date>2019-04-18T02:02:47.000Z/2019-04-18T02:11:37.000Z</dc:date>
    <ns3:datasetId>GEDI L4A Footprint Level Aboveground Biomass Density, Version 2.1</ns3:datasetId>
    <ns3:collectionConceptId>C2237824918-ORNL_CLOUD</ns3:collectionConceptId>
    <ns3:granuleSizeMB>113.29673</ns3:granuleSizeMB>
    <ns3:originalFormat>ECHO10</ns3:originalFormat>
    <ns3:dataCenter>ORNL_CLOUD</ns3:dataCenter>
    <ns3:coordinateSystem>GEODETIC</ns3:coordinateSystem>
    <ns6:polygon>-37.3154259 26.5281448 -34.8186836 29.9455051 -32.2175102 33.1447144 -29.5261726 36.1499557 -26.7580585 38.9866676 -23.9241676 41.6754875 -21.0334892 44.2404213 -18.0979824 46.6972198 -15.1244192 49.0658646 -12.6770945 50.943119 -12.6471701 50.9016037 -15.0940886 49.0242195 -18.0670738 46.6553917 -21.0018826 44.1984177 -23.891758 41.6333237 -26.72472 38.9444008 -29.4918041 36.1076546 -32.1819878 33.1025238 -34.7819061 29.9036064 -37.277462 26.4865437 -37.3154259 26.5281448</ns6:polygon>
  </ns0:entry>
  <ns0:entry>
    <ns0:id>https://cmr.earthdata.nasa.gov/opensearch/granules.atom?uid=G2613752613-ORNL_CLOUD</ns0:id>
    <ns0:title type="text">GEDI_L4A_AGB_Density_V2_1.GEDI04_A_2019108015253_O01960_02_T03910_02_002_02_V002.h5</ns0:title>
    <ns0:updated>2025-03-03T19:38:07.000Z</ns0:updated>
    <ns0:link rel="enclosure" title="Download GEDI04_A_2019108015253_O01960_02_T03910_02_002_02_V002.h5" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/protected/gedi/GEDI_L4A_AGB_Density_V2_1/data/GEDI04_A_2019108015253_O01960_02_T03910_02_002_02_V002.h5" />
    <ns0:link rel="http://esipfed.org/ns/fedsearch/1.1/s3#" title="This link provides direct download access via S3 to the granule" hreflang="en-US" href="s3://ornl-cumulus-prod-protected/gedi/GEDI_L4A_AGB_Density_V2_1/data/GEDI04_A_2019108015253_O01960_02_T03910_02_002_02_V002.h5" />
    <ns0:link rel="describedBy" title="ORNL DAAC Data Set Documentation (USER'S GUIDE)" hreflang="en-US" href="https://daac.ornl.gov/GEDI/guides/GEDI_L4A_AGB_Density_V2_1.html" type="text/html" />
    <ns0:link rel="via" title="Data set Landing Page DOI URL (DATA SET LANDING PAGE)" hreflang="en-US" href="https://doi.org/10.3334/ORNLDAAC/2056" />
    <ns0:link rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_ATBD_L4A_v1.0.pdf" type="application/pdf" />
    <ns0:link rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_L4A_AGB_Density_V2_1.pdf" type="application/pdf" />
    <ns0:link rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_L4A_V2_Common_Queries.pdf" type="application/pdf" />
    <ns0:link rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_L4A_V2_Product_Data_Dictionary.pdf" type="application/pdf" />
    <ns0:link rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_deleted_files_mw_163_182.txt" />
    <ns0:link rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_deleted_files_orbits_17591_09899.txt" />
    <ns0:link rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_revised_files_mw_163_182.txt" />
    <ns0:link rel="via" title="Data Set Documentation (GENERAL DOCUMENTATION)" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_revised_files_orbits_17591_09899.txt" />
    <ns0:link rel="via" title="api endpoint to retrieve temporary credentials valid for same-region direct s3 access (VIEW RELATED INFORMATION)" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/s3credentials" />
    <ns0:link rel="http://esipfed.org/ns/fedsearch/1.1/service#" title="OPeNDAP request URL (GET DATA : OPENDAP DATA)" hreflang="en-US" href="https://opendap.earthdata.nasa.gov/collections/C2237824918-ORNL_CLOUD/granules/GEDI_L4A_AGB_Density_V2_1.GEDI04_A_2019108015253_O01960_02_T03910_02_002_02_V002.h5" />
    <ns0:link ns3:inherited="true" rel="describedBy" hreflang="en-US" href="https://daac.ornl.gov/GEDI/guides/GEDI_L4A_AGB_Density_V2_1.html" type="text/html">
      <ns3:inherited />
    </ns0:link>
    <ns0:link ns3:inherited="true" rel="via" hreflang="en-US" href="https://doi.org/10.3334/ORNLDAAC/2056">
      <ns3:inherited />
    </ns0:link>
    <ns0:link ns3:inherited="true" rel="http://esipfed.org/ns/fedsearch/1.1/service#" hreflang="en-US" href="https://opendap.earthdata.nasa.gov/">
      <ns3:inherited />
    </ns0:link>
    <ns0:link ns3:inherited="true" rel="via" hreflang="en-US" href="https://gedi.umd.edu">
      <ns3:inherited />
    </ns0:link>
    <ns0:link ns3:inherited="true" rel="describedBy" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_ATBD_L4A_v1.0.pdf" type="application/pdf">
      <ns3:inherited />
    </ns0:link>
    <ns0:link ns3:inherited="true" rel="describedBy" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_L4A_V2_Common_Queries.pdf" type="application/pdf">
      <ns3:inherited />
    </ns0:link>
    <ns0:link ns3:inherited="true" rel="describedBy" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_L4A_AGB_Density_V2_1.pdf" type="application/pdf">
      <ns3:inherited />
    </ns0:link>
    <ns0:link ns3:inherited="true" rel="describedBy" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_deleted_files_mw_163_182.txt">
      <ns3:inherited />
    </ns0:link>
    <ns0:link ns3:inherited="true" rel="describedBy" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_revised_files_mw_163_182.txt">
      <ns3:inherited />
    </ns0:link>
    <ns0:link ns3:inherited="true" rel="describedBy" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_deleted_files_orbits_17591_09899.txt">
      <ns3:inherited />
    </ns0:link>
    <ns0:link ns3:inherited="true" rel="describedBy" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/GEDI_L4A_V2_Product_Data_Dictionary.pdf" type="application/pdf">
      <ns3:inherited />
    </ns0:link>
    <ns0:link ns3:inherited="true" rel="describedBy" hreflang="en-US" href="https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4A_AGB_Density_V2_1/comp/list_of_revised_files_orbits_17591_09899.txt">
      <ns3:inherited />
    </ns0:link>
    <ns0:link href="https://cmr.earthdata.nasa.gov/search/concepts/G2613752613-ORNL_CLOUD.xml" hreflang="en-US" type="application/xml" rel="via" title="Product metadata" />
    <dc:identifier>G2613752613-ORNL_CLOUD</dc:identifier>
    <dc:date>2019-04-18T02:15:57.000Z/2019-04-18T02:38:57.000Z</dc:date>
    <ns3:datasetId>GEDI L4A Footprint Level Aboveground Biomass Density, Version 2.1</ns3:datasetId>
    <ns3:collectionConceptId>C2237824918-ORNL_CLOUD</ns3:collectionConceptId>
    <ns3:granuleSizeMB>324.322707</ns3:granuleSizeMB>
    <ns3:originalFormat>ECHO10</ns3:originalFormat>
    <ns3:dataCenter>ORNL_CLOUD</ns3:dataCenter>
    <ns3:coordinateSystem>GEODETIC</ns3:coordinateSystem>
    <ns6:polygon>0.579059 60.4911842 3.642535 62.6533508 6.6990881 64.8318405 9.7432947 67.0409622 12.7691469 69.2953491 15.7702131 71.6097717 18.7392311 74.0002365 21.6690865 76.483963 24.5510254 79.0796356 27.376009 81.8069077 30.1319752 84.686348 32.8091354 87.7424774 35.3923035 90.9988632 37.8662415 94.4803543 40.213459 98.2141571 42.4124565 102.2218399 44.4413834 106.5232315 46.2767677 111.1362915 47.8905945 116.0625076 49.2572479 121.2932892 50.3497734 126.8029556 51.1454773 132.5445099 51.6247368 138.4529724 51.7762222 144.4246979 51.8265839 144.4247131 51.6749001 138.4457703 51.1950531 132.5303955 50.3984261 126.7825546 49.3046951 121.2674103 47.9366417 116.0320816 46.3212852 111.102211 44.484314 106.4863358 42.4537964 102.1828766 40.2532463 98.1737137 37.9045448 94.4389267 35.4292183 90.9568329 32.8447685 87.700119 30.1664371 84.6438751 27.4094162 81.7644653 24.5834961 79.0373077 21.7007351 76.441803 18.7701778 73.9582672 15.8005648 71.5680084 12.7990179 69.2537689 9.772789 66.9995422 6.728313 64.7905426 3.6715939 62.612133 0.608061 60.4500084 0.579059 60.4911842</ns6:polygon>
  </ns0:entry>
</ns0:feed>
```


## Available Granule Search Parameters


In [41]:
%%html

<table>

<tr><th>Request<br/>
Parameter</th><th align="left">
Description
</th><th>Value &
Cardinality<br/>
(M) = mandatory<br/>
(O) = optional
</th><th>
Type</th></tr>


<tr><td>datasetId</td><td align="left">Collection identifier</td><td>(M) Allowed value is IDN collection ID</td><td> cwic:datasetId</td></tr>

<tr><td>geoBox</td><td align="left">Returned granules will have a spatial extent overlapping this bounding box
</td><td>(O) Supported formats are in W,S,E,N<br/>
coordinate order:<br/>
W: WestBoundingLongitude<br/>
S: SouthBoundingLatitude<br/>
E: EastBoundingLongitude<br/>
N: NorthBoundingLatitude<br/>
All coordinates are in EPSG:4326</td><td>geo:box</td></tr>

<tr><td>timeStart</td><td align="left">Returned granules will have a 
temporal extent containing this start time
</td><td>(O) Supported formats are:
'yyyy-MM-dd', 'yyyy-MM￾ddTHH:mm:ssZ' or 'yyyy-MM-dd 
HH:mm:ss'</td><td> time:start</td></tr>

<tr><td>timeEnd</td><td align="left">Returned granules will have a 
temporal extent containing this end time
</td><td>(O) Supported formats are:
'yyyy-MM-dd', 'yyyy-MM￾ddTHH:mm:ssZ' or 'yyyy-MM-dd 
HH:mm:ss'</td><td> time:end</td></tr>


<tr><td>startIndex</td><td align="left">Starting record number of the 
set of search results desired by 
the search client</td><td>(O) Allowed value is any integer equal 
and greater than ‘0’.</td><td>os:startIndex</td></tr>

<tr><td>count</td><td align="left">Number of search results per 
page desired by the search client
</td><td>(O) Allowed value is any integer within 
the interval of [0,200].</td><td> os:count</td></tr>


</table>

Request Parameter,Description,Value & Cardinality (M) = mandatory (O) = optional,Type
datasetId,Collection identifier,(M) Allowed value is IDN collection ID,cwic:datasetId
geoBox,Returned granules will have a spatial extent overlapping this bounding box,"(O) Supported formats are in W,S,E,N coordinate order: W: WestBoundingLongitude S: SouthBoundingLatitude E: EastBoundingLongitude N: NorthBoundingLatitude All coordinates are in EPSG:4326",geo:box
timeStart,Returned granules will have a temporal extent containing this start time,"(O) Supported formats are: 'yyyy-MM-dd', 'yyyy-MM￾ddTHH:mm:ssZ' or 'yyyy-MM-dd HH:mm:ss'",time:start
timeEnd,Returned granules will have a temporal extent containing this end time,"(O) Supported formats are: 'yyyy-MM-dd', 'yyyy-MM￾ddTHH:mm:ssZ' or 'yyyy-MM-dd HH:mm:ss'",time:end
startIndex,Starting record number of the set of search results desired by the search client,(O) Allowed value is any integer equal and greater than ‘0’.,os:startIndex
count,Number of search results per page desired by the search client,"(O) Allowed value is any integer within the interval of [0,200].",os:count
